# 机器学习如何助力气候变化预估作业1

### 作业1：
这个作业是基础题目，是关于模型的理解和运用的。请你结合项目教学内容，自行学习并运行代码，熟悉掌握海温偏差订正的全过程。修改优化函数、激活函数和隐藏层数目以及神经元数目这些参数，可参考模型的中文说明，分别探究对结果的影响，最终得到这些参数的最优组合。具体修改建议如下：
   1）优化函数：即‘solver’，有‘lbfgs’, ‘sgd’, ‘adam’可选择。
   2）激活函数：即‘activation’，有‘identity’, ‘logistic’, ‘tanh’, ‘relu’可选择。
   3）隐藏层数目以及神经元数目：即‘hidden_layer_sizes = (number)’, 可设置2-3层，神经元数目可以上一章建模时候设置6-15调试。
调参时，可采用单一变量法，观察一个参数的变化对结果带来的影响，体验调参的乐趣。最终的最优结果能否实现对历史数据50%以上的订正效果甚至更高？

In [1]:
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset  # 读取nc文件用到的包
from scipy.interpolate import griddata  # 对SST空间插值用到的函数
import time
import matplotlib.pyplot as plt
import scipy.io as io
import os

In [2]:
Path_Data = '/home/mw/input/CMIP6_SST1789/'  # 读取数据路径
NC_ERSST = Dataset(Path_Data + 'sst.mnmean.nc') # 读取数据
print(NC_ERSST)  # 打印查看文件信息

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    climatology: Climatology is based on 1971-2000 SST, Xue, Y., T. M. Smith, and R. W. Reynolds, 2003: Interdecadal changes of 30-yr SST normals during 1871.2000. Journal of Climate, 16, 1601-1612.
    description: In situ data: ICOADS2.5 before 2007 and NCEP in situ data from 2008 to present. Ice data: HadISST ice before 2010 and NCEP ice after 2010.
    keywords_vocabulary: NASA Global Change Master Directory (GCMD) Science Keywords
    keywords: Earth Science > Oceans > Ocean Temperature > Sea Surface Temperature >
    instrument: Conventional thermometers
    source_comment: SSTs were observed by conventional thermometers in Buckets (insulated or un-insulated canvas and wooded buckets) or Engine Room Intaker
    geospatial_lon_min: -1.0
    geospatial_lon_max: 359.0
    geospatial_laty_max: 89.0
    geospatial_laty_min: -89.0
    geospatial_lat_max: 89.0
    geospatial_lat_min: -89.0
    

In [3]:
print(NC_ERSST.variables.keys()) # 查看一下有哪些变量

dict_keys(['lat', 'lon', 'time_bnds', 'time', 'sst'])


In [4]:
print(NC_ERSST.variables['sst'].missing_value ) # 查看一下变量缺失值

-9.96921e+36


In [5]:
print(NC_ERSST.variables['time']) # 查看一下变量时间信息

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: days since 1800-1-1 00:00:00
    long_name: Time
    delta_t: 0000-01-00 00:00:00
    avg_period: 0000-01-00 00:00:00
    prev_avg_period: 0000-00-07 00:00:00
    standard_name: time
    axis: T
    actual_range: [19723. 80322.]
unlimited dimensions: time
current shape = (1992,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [6]:
Lat_ERSST = NC_ERSST.variables['lat'][:].data  # 纬度，维度是1维的，范围是88 到 -88，即(89,)
Lon_ERSST = NC_ERSST.variables['lon'][:].data  # 经度，,维度是1维的，范围是0 到 358,即(180,)
# SST，维度是3维的，分别是(时间, 纬度, 经度)，即(1992, 89, 180)
SST_ERSST = NC_ERSST.variables['sst'][:].data
Time_ERSST0 = NC_ERSST.variables['time'][:]  # 时间，维度是1维的，即(1992,)
Time_ERSST0 = nc.num2date(
    Time_ERSST0,
    'days since 1800-1-1 00:00:00').data  # 185401-201912,共1992个元素
print(
    Lat_ERSST.shape,
    Lon_ERSST.shape,
    SST_ERSST.shape,
    len(Time_ERSST0))  # 打印各个变量的尺寸

(89,) (180,) (1992, 89, 180) 1992


In [7]:
# 自己造一个日期列表
Time_ERSST = [int(Time_ERSST0[i].year*100) + int(Time_ERSST0[i].month)
              for i in range(Time_ERSST0.shape[0])]  # 185401-201912, 共1992个元素
print(len(Time_ERSST))  # 查看Time_ERSST长度
print(Time_ERSST[0:5], Time_ERSST[-5:])  # 打印时间列表头尾5个内容

1992
[185401, 185402, 185403, 185404, 185405] [201908, 201909, 201910, 201911, 201912]


In [8]:
NC_Historical = Dataset(Path_Data + 'tos_Omon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc')
print(NC_Historical) # 打印查看文件信息

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.7 CMIP-6.2
    activity_id: CMIP
    branch_method: Standard
    branch_time_in_child: 0.0
    branch_time_in_parent: 2289.0
    comment: The model integration starts from the piControl experiment equilibrium state (1st Jan. of the year 2289)
    contact: Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date: 2018-11-26T09:02:53Z
    data_specs_version: 01.00.27
    description: DECK: historical
    experiment: all-forcing simulation of the recent past
    experiment_id: historical
    external_variables: areacello
    forcing_index: 1
    frequency: mon
    further_info_url: https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM2-MR.historical.none.r1i1p1f1
    grid: native ocean tri-polar grid
    grid_label: gn
    history: 2018-11-26T09:02:52Z ; CMOR rewrote data to be consistent with CMIP6, CF-1.7 CMIP-6.2 and CF standards.;
N/A
    initialization_index: 1
    institution: 

In [9]:
print(NC_Historical.variables.keys())  # 查看一下有哪些变量

dict_keys(['time', 'time_bnds', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'latitude', 'longitude', 'tos'])


In [10]:
print(NC_Historical.variables['tos'].missing_value ) # 查看一下变量缺失值

1e+20


In [11]:
print(NC_Historical.variables['time'])   # 查看一下变量时间信息

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    bounds: time_bnds
    units: days since 1850-1-1
    calendar: 365_day
    axis: T
    long_name: time
    standard_name: time
unlimited dimensions: time
current shape = (1980,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [12]:
# 纬度，是2维的网格，按照(纬度,经度)排列数组大小是(384, 320)
Lat_Historical = NC_Historical.variables['latitude'][:].data
# 经度，是2维的网格，按照(纬度,经度)排列数组大小是(384, 320)
Lon_Historical = NC_Historical.variables['longitude'][:].data
# SST，3维，按照(时间, 纬度, 经度)排列数组大小是(1980, 384, 320)
SST_Historical = NC_Historical.variables['tos'][:].data
# 自己造一个日期列表,185001-201412,共1980个元素。
Time_Historical = [int(i*100) + int(j) for i in range(1850, 2015)
                   for j in range(1, 13)]  # 185001-201412,共1980个元素。
print(
    Lat_Historical.shape,
    Lon_Historical.shape,
    SST_Historical.shape,
    len(Time_Historical))  # 打印各个变量的尺寸

(232, 360) (232, 360) (1980, 232, 360) 1980


In [13]:
## 未来预估低排放情景：SSP1-2.6
NC_SSP126 = Dataset(Path_Data + 'tos_Omon_BCC-CSM2-MR_ssp126_r1i1p1f1_gn_201501-210012.nc')
print(NC_SSP126)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.7 CMIP-6.2
    activity_id: ScenarioMIP
    branch_method: Standard
    branch_time_in_child: 0.0
    branch_time_in_parent: 2015.0
    comment: This is an extension of historical simulation (r1i1p1f1) with prescribed scenario of SSP1-2.6.
    contact: Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date: 2019-03-19T09:02:06Z
    data_specs_version: 01.00.27
    description: SSP1-2.6
    experiment: update of RCP2.6 based on SSP1
    experiment_id: ssp126
    external_variables: areacello
    forcing_index: 1
    frequency: mon
    further_info_url: https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM2-MR.ssp126.none.r1i1p1f1
    grid: native ocean tri-polar grid
    grid_label: gn
    history: 2019-03-19T09:02:06Z ; CMOR rewrote data to be consistent with CMIP6, CF-1.7 CMIP-6.2 and CF standards.
    initialization_index: 1
    institution: Beijing Climate Center, Beijing 100

In [14]:
print(NC_SSP126.variables.keys()) # 查看一下有哪些变量

dict_keys(['time', 'time_bnds', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'latitude', 'longitude', 'tos'])


In [15]:
print(NC_SSP126.variables['tos'].missing_value )  # 查看一下变量缺失值

1e+20


In [16]:
print(NC_SSP126.variables['time'])   # 查看一下变量时间信息

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    bounds: time_bnds
    units: days since 2015-1-1
    calendar: 365_day
    axis: T
    long_name: time
    standard_name: time
unlimited dimensions: time
current shape = (1032,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [17]:
# 纬度，同历史数据，是2维的网格，按照(纬度,经度)排列数组大小是(384, 320)
Lat_SSP = NC_SSP126.variables['latitude'][:].data
# 经度，同历史数据，是2维的网格，按照(纬度,经度)排列数组大小是(384, 320)
Lon_SSP = NC_SSP126.variables['longitude'][:].data
# SST，3维，按照(时间, 纬度, 经度)排列数组大小是(1032, 384, 320)
SST_SSP126 = NC_SSP126.variables['tos'][:].data

In [18]:
# 自己造一个日期列表
Time_SSP = [int(i*100) + int(j) for i in range(2015, 2101)
            for j in range(1, 13)]  # 201501-210012,(1032,)
print(Lat_SSP.shape, Lon_SSP.shape, SST_SSP126.shape, len(Time_SSP))

(232, 360) (232, 360) (1032, 232, 360) 1032


In [19]:
# 未来预估中等排放情景：SSP245，除了SST不同，其他都相同，我们只需读取SST即可
NC_SSP245 = Dataset(
    Path_Data +
    'tos_Omon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc')
SST_SSP245 = NC_SSP245.variables['tos'][:].data   # (1032, 384, 320)
# 未来预估高排放情景：SSP585，除了SST不同，其他都相同，我们只需读取SST即可
NC_SSP585 = Dataset(
    Path_Data +
    'tos_Omon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc')
SST_SSP585 = NC_SSP585.variables['tos'][:].data   # (1032, 384, 320)

In [20]:
# 设置经纬度最大最小值
Min_Lat_ERSST = -88
Max_Lat_ERSST = 88
Min_Lon_ERSST = 0
Max_Lon_ERSST = 359
# 插值前后的经纬度间隔
Step_LatLon_Before = 2
Step_LatLon_After = 1
# 给经纬度打网格
# 用到了meshgrid，将1维的经纬度信息生成二维的网格点坐标矩阵，插值处理和画图常用到。
Lat_ERSST_mgrid, Lon_ERSST_mgrid = np.meshgrid(Lat_ERSST, Lon_ERSST)
print(Lat_ERSST_mgrid.shape, Lon_ERSST_mgrid.shape)
# 经纬度的维度信息，按照（纬度，经度）的顺序排列
Lat_ERSST_mgrid = Lat_ERSST_mgrid.T
Lon_ERSST_mgrid = Lon_ERSST_mgrid.T

(180, 89) (180, 89)


In [21]:
# 插值前的经纬度, 维度是(180*89,2)
LatLon_ERSST_Before = np.hstack(
    (Lat_ERSST_mgrid.reshape(-1, 1), Lon_ERSST_mgrid.reshape(-1, 1)))  # 按水平方向进行叠加，形成两列，
# 插值前的SST，维度是(180*89,1)
SST_ERSST_Before = SST_ERSST.reshape((SST_ERSST.shape[0], -1, 1))
# 插值前的经纬度和SST有了，我们还需要给出插值后的经纬度信息
Lat_ERSST_After_1D = np.arange(
    Max_Lat_ERSST, Min_Lat_ERSST-1, -Step_LatLon_After)  # 生成插值后的纬度
Lon_ERSST_After_1D = np.arange(
    Min_Lon_ERSST,
    Max_Lon_ERSST+1,
    Step_LatLon_After)  # 生成插值后的经度
Lat_ERSST_After_2D, Lon_ERSST_After_2D = np.meshgrid(
    Lat_ERSST_After_1D, Lon_ERSST_After_1D)  # 打网格
print(Lat_ERSST_After_2D.shape, Lon_ERSST_After_2D.shape)
Lat_ERSST_After_2D = Lat_ERSST_After_2D.T  # 转置一下，为了和变量维度匹配
Lon_ERSST_After_2D = Lon_ERSST_After_2D.T

(360, 177) (360, 177)


In [22]:
# 使用griddata函数对SST插值
SST_ERSST_After = np.zeros([SST_ERSST.shape[0],
                            Lat_ERSST_After_2D.shape[0],
                            Lat_ERSST_After_2D.shape[1]])
# 对每一个月的全球观测资料插值，并⏲计时
start_time = time.time()
for t in range(SST_ERSST.shape[0]):
    # griddata函数插值，需要输入插值前的经纬度信息，插值前的变量，插值后的经纬度，插值方法，这里插值方法选择'nearest'方法。
    SST_ERSST_After_Now = griddata(LatLon_ERSST_Before,
                                   SST_ERSST_Before[t,
                                                    :,
                                                    :],
                                   (Lat_ERSST_After_2D,
                                    Lon_ERSST_After_2D),
                                   method='nearest').squeeze()
    SST_ERSST_After[t, :, :] = SST_ERSST_After_Now
    if t % 100 == 0:  # 每100个数打印显示一下
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")  # 打印看一下循环计算的用时

# 清空一下无关变量，后面不再用了
del LatLon_ERSST_Before
del SST_ERSST_Before

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
94.04753088951111s


In [23]:
## 设置经纬度最大最小值
Min_Lat_Historical = -88
Max_Lat_Historical = 88
Min_Lon_Historical = 0
## 插值前后的经纬度间隔
Max_Lon_Historical = 359
Step_LatLon_After = 1
## 给经纬度打网格
Lat_Historical_mgrid = Lat_Historical  
Lon_Historical_mgrid = Lon_Historical
## 插值前的经纬度, 维度是(384*320,2)
LatLon_Historical_Before = np.hstack( ( Lat_Historical_mgrid.reshape(-1,1), Lon_Historical_mgrid.reshape(-1,1) ) ) 
SST_Historical_Before = SST_Historical.reshape( (SST_Historical.shape[0],-1,1) ) 
Lat_Historical_After_1D = np.arange(Max_Lat_Historical,Min_Lat_Historical-1,-Step_LatLon_After)
Lon_Historical_After_1D = np.arange(Min_Lon_Historical,Max_Lon_Historical+1,Step_LatLon_After)
Lat_Historical_After_2D, Lon_Historical_After_2D = np.meshgrid(Lat_Historical_After_1D, Lon_Historical_After_1D)
print(Lat_Historical_After_2D.shape, Lon_Historical_After_2D.shape)
Lat_Historical_After_2D = Lat_Historical_After_2D.T 
Lon_Historical_After_2D = Lon_Historical_After_2D.T

(360, 177) (360, 177)


In [24]:
# 使用griddata函数对SST插值
SST_Historical_After = np.zeros([SST_Historical.shape[0],
                                 Lat_Historical_After_2D.shape[0],
                                 Lat_Historical_After_2D.shape[1]])
# 开始循环计算并计时
start_time = time.time()
for t in range(SST_Historical.shape[0]):
    # griddata函数插值，需要输入插值前的经纬度信息，插值前的变量，插值后的经纬度，插值方法，这里插值方法选择'nearest'方法。
    SST_Historical_After_Now = griddata(LatLon_Historical_Before,
                                        SST_Historical_Before[t,
                                                              :,
                                                              :],
                                        (Lat_Historical_After_2D,
                                         Lon_Historical_After_2D),
                                        method='nearest').squeeze()
    SST_Historical_After_Now[np.where(
        abs(SST_Historical_After_Now) >= 50)] == np.nan  # 大于50的值赋成nan值
    SST_Historical_After[t, :, :] = SST_Historical_After_Now
    if t % 100 == 0:  # 每100个数打印显示一下
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")  # 打印用时

# 清空一下无关变量，后面不再用了
del LatLon_Historical_Before
del SST_Historical_Before

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
205.9861023426056s


In [25]:
# 设置经纬度最大最小值
Min_Lat_SSP = -88
Max_Lat_SSP = 88
Min_Lon_SSP = 0
Max_Lon_SSP = 359
# 插值前后的经纬度间隔
Step_LatLon_After = 1
# 给经纬度打网格
Lat_SSP_mgrid = Lat_SSP
Lon_SSP_mgrid = Lon_SSP
# 插值前的经纬度, 维度是(384*320,2)
LatLon_SSP_Before = np.hstack(
    (Lat_SSP_mgrid.reshape(-1, 1), Lon_SSP_mgrid.reshape(-1, 1)))
SST_SSP126_Before = SST_SSP126.reshape((SST_SSP126.shape[0], -1, 1))
SST_SSP245_Before = SST_SSP245.reshape((SST_SSP245.shape[0], -1, 1))
SST_SSP585_Before = SST_SSP585.reshape((SST_SSP585.shape[0], -1, 1))
Lat_SSP_After_1D = np.arange(Max_Lat_SSP, Min_Lat_SSP-1, -Step_LatLon_After)
Lon_SSP_After_1D = np.arange(Min_Lon_SSP, Max_Lon_SSP+1, Step_LatLon_After)
Lat_SSP_After_2D, Lon_SSP_After_2D = np.meshgrid(
    Lat_SSP_After_1D, Lon_SSP_After_1D)
print(Lat_SSP_After_2D.shape, Lon_SSP_After_2D.shape)
Lat_SSP_After_2D = Lat_SSP_After_2D.T
Lon_SSP_After_2D = Lon_SSP_After_2D.T

(360, 177) (360, 177)


In [26]:
# 使用griddata函数对SST插值
SST_SSP126_After = np.zeros(
    [SST_SSP126.shape[0], Lat_SSP_After_2D.shape[0],  Lat_SSP_After_2D.shape[1]])
SST_SSP245_After = np.zeros(
    [SST_SSP245.shape[0], Lat_SSP_After_2D.shape[0],  Lat_SSP_After_2D.shape[1]])
SST_SSP585_After = np.zeros(
    [SST_SSP585.shape[0], Lat_SSP_After_2D.shape[0],  Lat_SSP_After_2D.shape[1]])
# 开始循环计算并计时
start_time = time.time()
for t in range(SST_SSP126.shape[0]):
    # griddata函数插值，需要输入插值前的经纬度信息，插值前的变量，插值后的经纬度，插值方法，这里插值方法选择'nearest'方法。
    # SSP1-2.6
    SST_SSP126_After_Now = griddata(LatLon_SSP_Before,
                                    SST_SSP126_Before[t,
                                                      :,
                                                      :],
                                    (Lat_SSP_After_2D,
                                     Lon_SSP_After_2D),
                                    method='nearest')
    SST_SSP126_After[t, :, :] = SST_SSP126_After_Now.squeeze()
    # SSP2-4.5
    SST_SSP245_After_Now = griddata(LatLon_SSP_Before,
                                    SST_SSP245_Before[t,
                                                      :,
                                                      :],
                                    (Lat_SSP_After_2D,
                                     Lon_SSP_After_2D),
                                    method='nearest')
    SST_SSP245_After[t, :, :] = SST_SSP245_After_Now.squeeze()
    # SSP5-8.5
    SST_SSP585_After_Now = griddata(LatLon_SSP_Before,
                                    SST_SSP585_Before[t,
                                                      :,
                                                      :],
                                    (Lat_SSP_After_2D,
                                     Lon_SSP_After_2D),
                                    method='nearest')
    SST_SSP585_After[t, :, :] = SST_SSP585_After_Now.squeeze()
    if t % 100 == 0:
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")

# 清空一下无关变量，后面不再用了
del LatLon_SSP_Before
del SST_SSP126_Before
del SST_SSP245_Before
del SST_SSP585_Before

0
100
200
300
400
500
600
700
800
900
1000
320.2872772216797s


In [27]:
# 定义求区域平均函数
def areamean2D(X, lat, lon):
    """
    本函数计算经纬度网格下某变量区域平均值。

    输入：
        X: 待求区域平均的变量，如SST;
        lat: 纬度；
        lon: 经度。
    输出：
        X_AreaMean: 输入变量X的区域平均值

    """
    import numpy as np

    Len_lat = lat.shape[0]
    Len_lon = lon.shape[0]
    lat_2D = lat.repeat(Len_lon).reshape((Len_lat, Len_lon))
    Weight = np.cos(np.pi/180*lat_2D)  # 关键是要计算不同纬度对应的权重
    # 设置一个较大的较为合理的值，小于这个值才是正常的SST值
    Weight_Ocean = Weight[np.where(abs(X) < 50)]
    X_Ocean = X[np.where(abs(X) < 50)]  # 取出是海洋的点
    X_AreaMean = sum(X_Ocean * Weight_Ocean) / sum(Weight_Ocean)  # 区域平均计算公式

    return X_AreaMean  # 返回计算结果

In [28]:
SST_ERSST_AreaMean = np.zeros(
    SST_ERSST_After.shape[0])  # 预定义一个空数组，以储存观测的全球平均SST。
# 开始对观测SST逐月计算全球平均值
start_time = time.time()
for t in range(SST_ERSST_After.shape[0]):
    # 调用函数areamean2D
    SST_ERSST_AreaMean[t] = areamean2D(
        SST_ERSST_After[t, :, :], Lat_ERSST_After_1D, Lon_ERSST_After_1D)
    if t % 100 == 0:  # 每100个数打印显示一下
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")
del SST_ERSST_After
del Lat_ERSST_After_1D
del Lon_ERSST_After_1D

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
17.92681884765625s


In [29]:
SST_Historical_AreaMean = np.zeros(SST_Historical_After.shape[0])
# 开始对模式的历史SST逐月计算全球平均值
start_time = time.time()
for t in range(SST_Historical_After.shape[0]):
    SST_Historical_AreaMean[t] = areamean2D(
        SST_Historical_After[t, :, :], Lat_Historical_After_1D, Lon_Historical_After_1D)
    if t % 100 == 0:  # 每100个数打印显示一下
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")
del SST_Historical_After
del Lat_Historical_After_1D
del Lon_Historical_After_1D

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
12.87173581123352s


In [30]:
SST_SSP126_AreaMean = np.zeros(SST_SSP126_After.shape[0])
SST_SSP245_AreaMean = np.zeros(SST_SSP245_After.shape[0])
SST_SSP585_AreaMean = np.zeros(SST_SSP585_After.shape[0])
start_time = time.time()
for t in range(SST_SSP126_After.shape[0]):
    SST_SSP126_AreaMean[t] = areamean2D(
        SST_SSP126_After[t, :, :], Lat_SSP_After_1D, Lon_SSP_After_1D)
    SST_SSP245_AreaMean[t] = areamean2D(
        SST_SSP245_After[t, :, :], Lat_SSP_After_1D, Lon_SSP_After_1D)
    SST_SSP585_AreaMean[t] = areamean2D(
        SST_SSP585_After[t, :, :], Lat_SSP_After_1D, Lon_SSP_After_1D)
    if t % 100 == 0:
        print(t)
end_time = time.time()
print(str(end_time - start_time) + "s")

0
100
200
300
400
500
600
700
800
900
1000
19.93260169029236s


In [31]:
# 设置一下保存路径以及文件夹存在与否的判断
Path_Pre = '/home/mw/project/'
Path_SaveData = Path_Pre + 'SaveData/Original/'
isExists = os.path.exists(Path_SaveData)
if not isExists:
    os.makedirs(Path_Pre + 'SaveData/Original')   # 创建文件夹
    # 创建文件夹，打印 ' successfully build!'
    print(Path_SaveData + ' successfully build!')
else:
    # 如果文件夹已经建立，打印 ‘has been existed!’
    print(Path_SaveData + ' has been existed!')

# 保存成npy文件
Num_Year = 2014-1854+1
Num_Sample_SST = Num_Year*12
SST_ERSST_AreaMean = SST_ERSST_AreaMean[0:Num_Sample_SST]
SST_Historical_AreaMean = SST_Historical_AreaMean[-Num_Sample_SST::]
Time_ERSST = Time_ERSST[0:Num_Sample_SST]
np.save(Path_SaveData + 'SST_ERSST_AreaMean.npy', SST_ERSST_AreaMean)
np.save(Path_SaveData + 'SST_Historical_AreaMean.npy', SST_Historical_AreaMean)
np.save(Path_SaveData + 'SST_SSP126_AreaMean.npy', SST_SSP126_AreaMean)
np.save(Path_SaveData + 'SST_SSP245_AreaMean.npy', SST_SSP245_AreaMean)
np.save(Path_SaveData + 'SST_SSP585_AreaMean.npy', SST_SSP585_AreaMean)
np.save(Path_SaveData + 'Time_ERSST.npy', Time_ERSST)
np.save(Path_SaveData + 'Time_SSP.npy', Time_SSP)

# 保存成mat文件，留作后续octave环境下eemd数据分解使用
io.savemat(Path_SaveData + 'SST_ERSST_AreaMean.mat',
           {'SST_ERSST_AreaMean': SST_ERSST_AreaMean})
io.savemat(Path_SaveData + 'SST_Historical_AreaMean.mat',
           {'SST_Historical_AreaMean': SST_Historical_AreaMean})
io.savemat(Path_SaveData + 'SST_SSP126_AreaMean.mat',
           {'SST_SSP126_AreaMean': SST_SSP126_AreaMean})
io.savemat(Path_SaveData + 'SST_SSP245_AreaMean.mat',
           {'SST_SSP245_AreaMean': SST_SSP245_AreaMean})
io.savemat(Path_SaveData + 'SST_SSP585_AreaMean.mat',
           {'SST_SSP585_AreaMean': SST_SSP585_AreaMean})

/home/mw/project/SaveData/Original/ successfully build!


In [32]:
Path_Pre = '/home/mw/project/'
Path_SaveFigures = Path_Pre + 'Figures/Original/'
isExists = os.path.exists(Path_SaveFigures)
if not isExists:
    os.makedirs(Path_Pre + 'Figures/Original')
    print(Path_SaveFigures + ' successfully build!')
else:
    print(Path_SaveFigures + ' has been existed!')

/home/mw/project/Figures/Original/ successfully build!


In [33]:
# 先预设一下画图的设置
markersizenum = 4  # 标记大小
Num_Year = 2014-1854+1
Num_Sample_SST = Num_Year*12
X_Time = np.arange(0, Num_Sample_SST)  # 生成时间数组
Time_SST = np.array(Time_Historical[-Num_Sample_SST::])
xtick_SST_interval = 20*12  # 画图横坐标的刻度间隔
# SST
SST_Historical_AreaMean = SST_Historical_AreaMean[-Num_Sample_SST::]
SST_ERSST_AreaMean = SST_ERSST_AreaMean[0:Num_Sample_SST]
xticklabel_SST = list(range(0, Num_Sample_SST, xtick_SST_interval))
print(xticklabel_SST)
xticklabels_SST = Time_SST[xticklabel_SST[0::]].astype('str').tolist()
ylim_sst_max = 19
ylim_sst_min = 16
delta_ylim_sst = ylim_sst_max - ylim_sst_min
yticklabel_SST = np.linspace(ylim_sst_min, ylim_sst_max, num=7).tolist()
yticklabel_SST = [round(i, 1) for i in yticklabel_SST]
print(yticklabel_SST)
yticklabels_SST = [str(i) for i in yticklabel_SST]
print(yticklabels_SST)
# SST偏差
SST_Bias_AreaMean = SST_Historical_AreaMean - SST_ERSST_AreaMean
xticklabel_Bias = xticklabel_SST
xticklabels_Bias = xticklabels_SST
ylim_bias_max = 1
ylim_bias_min = -1
delta_ylim_bias = ylim_bias_max - ylim_bias_min
yticklabel_Bias = np.linspace(-1, 1, num=5).tolist()
yticklabel_Bias = [round(i, 1) for i in yticklabel_Bias]
print(yticklabel_Bias)
yticklabels_Bias = [str(i) for i in yticklabel_Bias]
print(yticklabels_Bias)
str_panels = ['a)ERSST ', 'b)Historical', 'c)SST Bias ']
# 画图
fig, ax = plt.subplots(3, 1, figsize=(16, 9), dpi=600)
# 观测：ERSST
ax = plt.subplot(3, 1, 1)
ax.plot(
    X_Time,
    SST_ERSST_AreaMean,
    'k-',
    linewidth=2,
    marker='.',
    markersize=markersizenum,
    mfc='black')
ax.set_xlim(0, Num_Sample_SST)
ax.set_ylim(ylim_sst_min, ylim_sst_max)
ax.set_xticks(xticklabel_SST)
ax.set_yticks(yticklabel_SST)
ax.set_xticklabels(xticklabels_SST)
ax.set_yticklabels(yticklabels_SST)
ax.text(1, ylim_sst_min + 0.9*delta_ylim_sst, str_panels[0])
ax.set_ylabel('SST(℃)')
ax.grid(linestyle='-.')
# 模式：历史时期
ax = plt.subplot(3, 1, 2)
ax.plot(
    X_Time,
    SST_Historical_AreaMean,
    'b-',
    linewidth=2,
    marker='.',
    markersize=markersizenum,
    mfc='blue')
ax.set_xlim(0, Num_Sample_SST)
ax.set_ylim(ylim_sst_min, ylim_sst_max)
ax.set_xticks(xticklabel_SST)
ax.set_yticks(yticklabel_SST)
ax.set_xticklabels(xticklabels_SST)
ax.set_yticklabels(yticklabels_SST)
ax.text(1, ylim_sst_min + 0.9*delta_ylim_sst, str_panels[1])
ax.set_ylabel('SST(℃)')
ax.grid(linestyle='-.')
# 模式偏差：模式减观测
ax = plt.subplot(3, 1, 3)
ax.plot(
    X_Time,
    SST_Bias_AreaMean,
    'b-',
    linewidth=2,
    marker='.',
    markersize=markersizenum,
    mfc='blue')
ax.set_xlim(0, Num_Sample_SST)
ax.set_ylim(ylim_bias_min, ylim_bias_max)
ax.set_xticks(xticklabel_Bias)
ax.set_yticks(yticklabel_Bias)
ax.set_xticklabels(xticklabels_Bias)
ax.set_yticklabels(yticklabels_Bias)
ax.text(1, ylim_bias_min + 0.9*delta_ylim_bias, str_panels[2])
ax.set_xlabel('Time')
ax.set_ylabel('SST Bias(℃)')
ax.grid(linestyle='-.')
# 保存图片
fns = os.path.join(Path_SaveFigures, 'SST_ERSST_Historical_Bias.png')
plt.savefig(fns, dpi=600, bbox_inches='tight')
plt.show()
plt.close()

[0, 240, 480, 720, 960, 1200, 1440, 1680, 1920]
[16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0]
['16.0', '16.5', '17.0', '17.5', '18.0', '18.5', '19.0']
[-1.0, -0.5, 0.0, 0.5, 1.0]
['-1.0', '-0.5', '0.0', '0.5', '1.0']


<Figure size 9600x5400 with 3 Axes>

# EEMD分解部分见EEMD.IPYNB文件

## EEMD后处理

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import os

In [2]:
# 原始数据
Path_Data_Original = '/home/mw/project/SaveData/Original/'
Time_ERSST = np.load(Path_Data_Original + 'Time_ERSST.npy')
Time_SSP = np.load(Path_Data_Original + 'Time_SSP.npy')
Num_Year = 86  # 1929-2014
Num_Sample_SST = Num_Year*12
Time_SST = np.arange(0,Num_Sample_SST)
Time_ERSST = Time_ERSST[-Num_Sample_SST::]
# EEMD分解的数据
Path_Data_EEMD = '/home/mw/project/SaveData/EEMD/'
IMFs_ERSST = sio.loadmat(Path_Data_EEMD + 'IMFs_ERSST.mat'); IMFs_ERSST = IMFs_ERSST['IMFs_ERSST']
IMFs_Historical = sio.loadmat(Path_Data_EEMD + 'IMFs_Historical.mat'); IMFs_Historical = IMFs_Historical['IMFs_Historical']
IMFs_SSP126 = sio.loadmat(Path_Data_EEMD + 'IMFs_SSP126.mat'); IMFs_SSP126 = IMFs_SSP126['IMFs_SSP126']
IMFs_SSP245 = sio.loadmat(Path_Data_EEMD + 'IMFs_SSP245.mat'); IMFs_SSP245 = IMFs_SSP245['IMFs_SSP245']
IMFs_SSP585 = sio.loadmat(Path_Data_EEMD + 'IMFs_SSP585.mat'); IMFs_SSP585 = IMFs_SSP585['IMFs_SSP585']
# 保存成npy文件
np.save(Path_Data_EEMD + 'IMFs_ERSST.npy', IMFs_ERSST)
np.save(Path_Data_EEMD + 'IMFs_Historical.npy', IMFs_Historical)
np.save(Path_Data_EEMD + 'IMFs_SSP126.npy', IMFs_SSP126)
np.save(Path_Data_EEMD + 'IMFs_SSP245.npy', IMFs_SSP245)
np.save(Path_Data_EEMD + 'IMFs_SSP585.npy', IMFs_SSP585)

In [3]:
Path_Pre = '/home/mw/project/Figures/'
Path_SaveFigures =  Path_Pre + 'EEMD/'
isExists = os.path.exists(Path_SaveFigures)
if not isExists:
    os.makedirs(Path_Pre + 'EEMD/')
    print( Path_SaveFigures +  ' successfully build!')
else:
    print( Path_SaveFigures + ' has been existed!')

/home/mw/project/Figures/EEMD/ successfully build!


In [4]:
NIMFs_ERSST = IMFs_ERSST.shape[0]
Max_IMFs_ERSST = np.max(IMFs_ERSST, 1)
Min_IMFs_ERSST = np.min(IMFs_ERSST, 1)
# 预设
fontsizenum = 16
Num_Year = 86
Num_Sample_SST = Num_Year*12
X_Time = np.arange(0, Num_Sample_SST)
Time_SST = np.array(Time_SST[-Num_Sample_SST::])
xtick_SST_interval = 7*12
xticklabel_SST = list(range(0, Num_Sample_SST, xtick_SST_interval))
xticklabels_SST = Time_ERSST[xticklabel_SST[0::]].astype('str').tolist()
# 画图
fig, ax = plt.subplots(NIMFs_ERSST, 1, figsize=(15, 18), dpi=600)
for n in range(NIMFs_ERSST):
    ax = plt.subplot(NIMFs_ERSST, 1, n + 1)
    ax.plot(X_Time, IMFs_ERSST[n, :], 'k-', linewidth=2)
    ax.set_xlim(0, Num_Sample_SST)
    ax.set_xticks(xticklabel_SST)
    ax.set_xticklabels([])
    yticks_all = np.round(ax.get_yticks(), 1)
    if yticks_all[0] == yticks_all[-1]:
        yticks_all[0] = -0.1
        yticks_all[-1] = 0.1
    yticks_end = np.array((yticks_all[0], np.round(
        (yticks_all[0] + yticks_all[-1]) / 2, 1), yticks_all[-1]))
    yticklabels_end = [str(i) for i in yticks_end]
    print(yticklabels_end)
    ax.set_yticks(yticks_end)
    ax.set_yticklabels(yticklabels_end, fontsize=fontsizenum)
    ax.set_ylabel('IMF' + str(n+1), fontsize=fontsizenum)
ax.set_xticks(xticklabel_SST)
ax.set_xticklabels(xticklabels_SST, fontsize=fontsizenum)
ax.set_xlabel('Time', fontsize=fontsizenum)
# 保存图片，先保存再显示。
fns = os.path.join(Path_SaveFigures, 'IMFs_ERSST.png')
plt.savefig(fns, dpi=600, bbox_inches='tight')
plt.show()  # 显示图片
plt.close()

['-0.3', '-0.0', '0.2']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '-0.0', '0.0']
['-0.1', '0.0', '0.1']
['17.6', '18.0', '18.4']


<Figure size 9000x10800 with 10 Axes>

In [5]:
NIMFs_Historical = IMFs_Historical.shape[0]
Max_IMFs_Historical = np.max(IMFs_Historical, 1)
Min_IMFs_Historical = np.min(IMFs_Historical, 1)
# 预设
fontsizenum = 16  
Num_Year = 86
Num_Sample_SST = Num_Year*12
X_Time = np.arange(0, Num_Sample_SST )
Time_SST = np.array(Time_SST[-Num_Sample_SST::])
xtick_SST_interval = 7*12
xticklabel_SST = list(range(0, Num_Sample_SST, xtick_SST_interval))
xticklabels_SST = Time_ERSST[xticklabel_SST[0::]].astype('str').tolist()
# 画图
fig, ax = plt.subplots(NIMFs_Historical, 1, figsize = (15,18), dpi = 600)
for n in range(NIMFs_Historical):
    ax = plt.subplot(NIMFs_Historical, 1, n + 1)
    ax.plot(X_Time, IMFs_Historical[n,:], 'b-', linewidth = 2)
    ax.set_xlim(0, Num_Sample_SST)
    ax.set_xticks(xticklabel_SST)
    ax.set_xticklabels([])
    yticks_all = np.round(ax.get_yticks(), 1)
    if yticks_all[0] == yticks_all[-1]:
        yticks_all[0] = -0.1
        yticks_all[-1] = 0.1
    yticks_end = np.array((yticks_all[0], np.round((yticks_all[0] + yticks_all[-1]) / 2, 1), yticks_all[-1]))
    yticklabels_end = [str(i) for i in yticks_end]
    print(yticklabels_end)
    ax.set_yticks(yticks_end)
    ax.set_yticklabels(yticklabels_end, fontsize = fontsizenum)
    ax.set_ylabel('IMF' + str(n+1), fontsize = fontsizenum)
ax.set_xticks(xticklabel_SST)
ax.set_xticklabels(xticklabels_SST, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize = fontsizenum)
# 保存图片，先保存再显示。
fns = os.path.join(Path_SaveFigures, 'IMFs_Historical.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight') 
plt.show()  # 显示图片
plt.close()

['-1.5', '-0.2', '1.0']
['-2.0', '0.0', '2.0']
['-0.2', '0.0', '0.2']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['19.6', '20.0', '20.4']


<Figure size 9000x10800 with 10 Axes>

In [6]:
NIMFs_SSP = IMFs_SSP126.shape[0]
Max_IMFs_SSP126 = np.max(IMFs_SSP126, 1)
Min_IMFs_SSP126 = np.min(IMFs_SSP126, 1)
# 预设
fontsizenum = 16  
Num_Year = 86
Num_Sample_SST = Num_Year*12
X_Time = np.arange(0, Num_Sample_SST )
Time_SST = np.array(Time_SST[-Num_Sample_SST::])
xtick_SST_interval = 7*12
xticklabel_SST = list(range(0, Num_Sample_SST, xtick_SST_interval))
xticklabels_SST = Time_SSP[xticklabel_SST[0::]].astype('str').tolist()
# 画图
fig, ax = plt.subplots(NIMFs_SSP, 1, figsize = (15,18), dpi = 600)
for n in range(NIMFs_SSP):
    ax = plt.subplot(NIMFs_SSP, 1, n + 1)
    ax.plot(X_Time, IMFs_SSP126[n,:], 'b-', linewidth = 2)
    ax.set_xlim(0, Num_Sample_SST)
    ax.set_xticks(xticklabel_SST)
    ax.set_xticklabels([])
    yticks_all = np.round(ax.get_yticks(), 1)
    if yticks_all[0] == yticks_all[-1]:
        yticks_all[0] = -0.1
        yticks_all[-1] = 0.1
    yticks_end = np.array( (yticks_all[0], np.round((yticks_all[0] + yticks_all[-1]) / 2, 1), yticks_all[-1]) )
    yticklabels_end = [str(i) for i in yticks_end]
    print(yticklabels_end)
    ax.set_yticks(yticks_end)
    ax.set_yticklabels(yticklabels_end, fontsize = fontsizenum)
    ax.set_ylabel('IMF' + str(n+1), fontsize = fontsizenum)
ax.set_xticks(xticklabel_SST)
ax.set_xticklabels(xticklabels_SST, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize = fontsizenum)
# 保存图片，先保存再显示。
fns = os.path.join(Path_SaveFigures, 'IMFs_SSP126.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight') 
plt.show()  # 显示图片
plt.close()

['-1.0', '0.0', '1.0']
['-2.0', '0.0', '2.0']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.2']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.0', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['20.4', '20.7', '21.0']


<Figure size 9000x10800 with 10 Axes>

In [7]:
Max_IMFs_SSP245 = np.max(IMFs_SSP245, 1)
Min_IMFs_SSP245 = np.min(IMFs_SSP245, 1)
# 画图
fig, ax = plt.subplots(NIMFs_SSP, 1, figsize = (15,18), dpi = 600)
for n in range(NIMFs_SSP):
    ax = plt.subplot(NIMFs_SSP, 1, n + 1)
    ax.plot(X_Time, IMFs_SSP245[n,:], 'b-', linewidth = 2)
    ax.set_xlim(0, Num_Sample_SST)
    ax.set_xticks(xticklabel_SST)
    ax.set_xticklabels([])
    yticks_all = np.round(ax.get_yticks(), 1)
    if yticks_all[0] == yticks_all[-1]:
        yticks_all[0] = -0.1
        yticks_all[-1] = 0.1
    yticks_end = np.array( (yticks_all[0], np.round((yticks_all[0] + yticks_all[-1]) / 2, 1), yticks_all[-1]) )
    yticklabels_end = [str(i) for i in yticks_end]
    print(yticklabels_end)
    ax.set_yticks(yticks_end)
    ax.set_yticklabels(yticklabels_end, fontsize = fontsizenum)
    ax.set_ylabel('IMF' + str(n+1), fontsize = fontsizenum)
ax.set_xticks(xticklabel_SST)
ax.set_xticklabels(xticklabels_SST, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize = fontsizenum)
# 保存图片，先保存再显示。
fns = os.path.join(Path_SaveFigures, 'IMFs_SSP245.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight') 
plt.show()  # 显示图片
plt.close()

['-1.0', '0.5', '2.0']
['-2.0', '0.0', '2.0']
['-0.2', '0.0', '0.2']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['-0.0', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['20.0', '20.8', '21.5']


<Figure size 9000x10800 with 10 Axes>

In [8]:
Max_IMFs_SSP585 = np.max(IMFs_SSP585, 1)
Min_IMFs_SSP585 = np.min(IMFs_SSP585, 1)
# 画图
fig, ax = plt.subplots(NIMFs_SSP, 1, figsize = (15,18), dpi = 600)
for n in range(NIMFs_SSP):
    ax = plt.subplot(NIMFs_SSP, 1, n + 1)
    ax.plot(X_Time, IMFs_SSP585[n,:], 'b-', linewidth = 2)
    ax.set_xlim(0, Num_Sample_SST)
    ax.set_xticks(xticklabel_SST)
    ax.set_xticklabels([])
    yticks_all = np.round(ax.get_yticks(), 1)
    if yticks_all[0] == yticks_all[-1]:
        yticks_all[0] = -0.1
        yticks_all[-1] = 0.1
    yticks_end = np.array( (yticks_all[0], np.round((yticks_all[0] + yticks_all[-1]) / 2, 1), yticks_all[-1]) )
    yticklabels_end = [str(i) for i in yticks_end]
    print(yticklabels_end)
    ax.set_yticks(yticks_end)
    ax.set_yticklabels(yticklabels_end, fontsize = fontsizenum)
    ax.set_ylabel('IMF' + str(n+1), fontsize = fontsizenum)
ax.set_xticks(xticklabel_SST)
ax.set_xticklabels(xticklabels_SST, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize = fontsizenum)
# 保存图片，先保存再显示。
fns = os.path.join(Path_SaveFigures, 'IMFs_SSP585.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight') 
plt.show()  # 显示图片
plt.close()

['-1.0', '0.0', '1.0']
['-2.0', '0.0', '2.0']
['-0.4', '0.0', '0.4']
['-0.2', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.2']
['-0.1', '0.0', '0.1']
['-0.1', '0.0', '0.1']
['20.0', '21.5', '23.0']


<Figure size 9000x10800 with 10 Axes>

In [9]:
# ERSST
Num_CrossZero_ERSST = np.zeros(NIMFs_ERSST - 1)
MeanPeriod_IMFs_ERSST = np.zeros(NIMFs_ERSST - 1)
MeanYearPeriod_IMFs_ERSST = np.zeros(NIMFs_ERSST - 1)
for n in range(NIMFs_ERSST - 1):
    k = 0
    for i in range(len(Time_SST) - 1):
        if IMFs_ERSST[n,i] * IMFs_ERSST[n,i + 1] < 0:  # 数跨零点的个数
            k = k + 1
    Num_CrossZero_ERSST[n] = k
    MeanPeriod_IMFs_ERSST[n] = len(Time_SST) * 2 / k  # 根据跨零点的个数计算平均周期，单位是月
    MeanYearPeriod_IMFs_ERSST[n] = len(Time_SST) * 2 / k / 12  # 单位转换成年
# 打印观测的SST各个imf的平均周期
print(MeanYearPeriod_IMFs_ERSST)

# Historical
NIMFs_Historical = IMFs_Historical.shape[0]
Num_CrossZero_Historical = np.zeros(NIMFs_Historical - 1)
MeanPeriod_IMFs_Historical = np.zeros(NIMFs_Historical - 1)
MeanYearPeriod_IMFs_Historical = np.zeros(NIMFs_Historical - 1)
for n in range(NIMFs_Historical - 1):
    k = 0
    for i in range(len(Time_SST)-1):
        if IMFs_Historical[n,i] * IMFs_Historical[n,i + 1] < 0:
            k = k + 1
    Num_CrossZero_Historical[n] = k
    MeanPeriod_IMFs_Historical[n] = len(Time_SST) * 2 / k
    MeanYearPeriod_IMFs_Historical[n] = len(Time_SST) * 2 / k / 12
# 打印模式的历史SST各个imf的平均周期
print(MeanYearPeriod_IMFs_Historical)

# SSP126/SSP245/SSP585
NIMFs_SSP = IMFs_SSP126.shape[0]
Num_CrossZero_SSP126 = np.zeros(NIMFs_SSP - 1)
Num_CrossZero_SSP245 = np.zeros(NIMFs_SSP - 1)
Num_CrossZero_SSP585 = np.zeros(NIMFs_SSP - 1)
MeanPeriod_IMFs_SSP126 = np.zeros(NIMFs_SSP - 1)
MeanPeriod_IMFs_SSP245 = np.zeros(NIMFs_SSP - 1)
MeanPeriod_IMFs_SSP585 = np.zeros(NIMFs_SSP - 1)
MeanYearPeriod_IMFs_SSP126 = np.zeros(NIMFs_SSP - 1)
MeanYearPeriod_IMFs_SSP245 = np.zeros(NIMFs_SSP - 1)
MeanYearPeriod_IMFs_SSP585 = np.zeros(NIMFs_SSP - 1)
for n in range(NIMFs_SSP - 1):
    # SSP126
    k = 0
    for i in range(len(Time_SST)-1):
        if IMFs_SSP126[n,i] * IMFs_SSP126[n,i + 1] < 0:
            k = k + 1
    Num_CrossZero_SSP126[n] = k
    MeanPeriod_IMFs_SSP126[n] = len(Time_SST) * 2 / k
    MeanYearPeriod_IMFs_SSP126[n] = len(Time_SST) * 2 / k / 12

    # SSP245
    k = 0
    for i in range(len(Time_SST) - 1):
        if IMFs_SSP245[n, i] * IMFs_SSP245[n, i + 1] < 0:
            k = k + 1
    Num_CrossZero_SSP245[n] = k
    MeanPeriod_IMFs_SSP245[n] = len(Time_SST) * 2 / k
    MeanYearPeriod_IMFs_SSP245[n] = len(Time_SST) * 2 / k / 12

    # SSP585
    k = 0
    for i in range(len(Time_SST) - 1):
        if IMFs_SSP585[n, i] * IMFs_SSP585[n, i + 1] < 0:
            k = k + 1
    Num_CrossZero_SSP585[n] = k
    MeanPeriod_IMFs_SSP585[n] = len(Time_SST) * 2 / k
    MeanYearPeriod_IMFs_SSP585[n] = len(Time_SST) * 2 / k / 12
# 打印模式的未来预估SST各个imf的平均周期
print(MeanYearPeriod_IMFs_SSP126)
print(MeanYearPeriod_IMFs_SSP245)
print(MeanYearPeriod_IMFs_SSP585)

[ 0.49142857  0.86868687  1.12418301  3.37254902  6.61538462 13.23076923
 28.66666667 43.         86.        ]
[ 0.96089385  1.00584795  2.26315789  4.41025641  7.47826087 24.57142857
 34.4        86.         86.        ]
[ 0.97727273  1.          2.20512821  4.41025641  9.55555556 21.5
 86.         86.         86.        ]
[ 0.98850575  1.00584795  2.12345679  5.21212121  7.47826087 17.2
 34.4        86.         86.        ]
[ 0.92972973  1.00584795  2.23376623  4.0952381   7.47826087 24.57142857
 86.         86.         86.        ]


In [10]:
# 根据平均周期组合IMF
# ERSST
IMFs_ERSST_Compose = np.zeros( (6, Num_Sample_SST) )
IMFs_ERSST_Compose[0,:] = IMFs_ERSST[0,:]  # 第1个imf
IMFs_ERSST_Compose[1,:] = np.sum( IMFs_ERSST[1:3,:], 0 )  # 第2-3个imf
IMFs_ERSST_Compose[2,:] = np.sum( IMFs_ERSST[3:5,:], 0 )  # 第4-5个imf
IMFs_ERSST_Compose[3,:] = IMFs_ERSST[5,:]  # 第6个imf
IMFs_ERSST_Compose[4,:] = np.sum( IMFs_ERSST[6:8,:], 0 )  # 第7-8个imf
IMFs_ERSST_Compose[5,:] = np.sum( IMFs_ERSST[8:10,:], 0 ) # 第9个imf和最后一个趋势项
# Historical
IMFs_Historical_Compose = np.zeros( (6, Num_Sample_SST) )
IMFs_Historical_Compose[0,:] = IMFs_Historical[0,:] 
IMFs_Historical_Compose[1,:] = np.sum( IMFs_Historical[1:3,:], 0 )  
IMFs_Historical_Compose[2,:] = np.sum( IMFs_Historical[3:5,:], 0 )  
IMFs_Historical_Compose[3,:] = IMFs_Historical[5,:]  
IMFs_Historical_Compose[4,:] = np.sum( IMFs_Historical[6:8,:], 0 )
IMFs_Historical_Compose[5,:] = np.sum( IMFs_Historical[8:10,:], 0 )  

## SSP126/SSP245/SSP585
# SSP126
IMFs_SSP126_Compose = np.zeros( (6, Num_Sample_SST) )
IMFs_SSP126_Compose[0,:] = IMFs_SSP126[0,:]  
IMFs_SSP126_Compose[1,:] = np.sum( IMFs_SSP126[1:3,:], 0 ) 
IMFs_SSP126_Compose[2,:] = np.sum( IMFs_SSP126[3:5,:], 0 )  
IMFs_SSP126_Compose[3,:] = IMFs_SSP126[5,:]  
IMFs_SSP126_Compose[4,:] = np.sum( IMFs_SSP126[6:8,:], 0 )  
IMFs_SSP126_Compose[5,:] = np.sum( IMFs_SSP126[8:10,:], 0 )  
# SSP245
IMFs_SSP245_Compose = np.zeros( (6, Num_Sample_SST) )
IMFs_SSP245_Compose[0,:] = IMFs_SSP245[0,:] 
IMFs_SSP245_Compose[1,:] = np.sum( IMFs_SSP245[1:3,:], 0 )  
IMFs_SSP245_Compose[2,:] = np.sum( IMFs_SSP245[3:5,:], 0 )  
IMFs_SSP245_Compose[3,:] = IMFs_SSP245[5,:]  
IMFs_SSP245_Compose[4,:] = np.sum( IMFs_SSP245[6:8,:], 0 )  
IMFs_SSP245_Compose[5,:] = np.sum( IMFs_SSP245[8:10,:], 0 )  
# SSP585
IMFs_SSP585_Compose = np.zeros( (6, Num_Sample_SST) )
IMFs_SSP585_Compose[0,:] = IMFs_SSP585[0,:]  
IMFs_SSP585_Compose[1,:] = np.sum( IMFs_SSP585[1:3,:], 0 )  
IMFs_SSP585_Compose[2,:] = np.sum( IMFs_SSP585[3:5,:], 0 )  
IMFs_SSP585_Compose[3,:] = IMFs_SSP585[5,:]  
IMFs_SSP585_Compose[4,:] = np.sum( IMFs_SSP585[6:8,:], 0 )  
IMFs_SSP585_Compose[5,:] = np.sum( IMFs_SSP585[8:10,:], 0 )  

In [11]:
# 保存数据
Path_SaveData = '/home/mw/project/SaveData/EEMD/'
np.save(Path_SaveData + 'IMFs_ERSST_Compose.npy', IMFs_ERSST_Compose)
np.save(Path_SaveData + 'IMFs_Historical_Compose.npy', IMFs_Historical_Compose)
np.save(Path_SaveData + 'IMFs_SSP126_Compose.npy', IMFs_SSP126_Compose)
np.save(Path_SaveData + 'IMFs_SSP245_Compose.npy', IMFs_SSP245_Compose)
np.save(Path_SaveData + 'IMFs_SSP585_Compose.npy', IMFs_SSP585_Compose)

## SST订正及预估

In [2]:
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler  # 最大最小归一化函数
from sklearn.neural_network import MLPRegressor # 多层感知器回归，这里等于BPNN
import time
import pickle
import os
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression # 线性回归函数用于最后的海温增加量
 
# 定义均方根误差rmse函数
def rmse(predictions, targets):
    return np.sqrt( ( (predictions - targets)**2 ).mean() )

In [3]:
# EEMD分解后的组合数据
Path_EEMDData = '/home/mw/project/SaveData/EEMD/'
IMFs_ERSST_Compose = np.load(Path_EEMDData + 'IMFs_ERSST_Compose.npy')
IMFs_Historical_Compose = np.load(Path_EEMDData + 'IMFs_Historical_Compose.npy')
IMFs_SSP126_Compose = np.load(Path_EEMDData + 'IMFs_SSP126_Compose.npy')
IMFs_SSP245_Compose = np.load(Path_EEMDData + 'IMFs_SSP245_Compose.npy')
IMFs_SSP585_Compose = np.load(Path_EEMDData + 'IMFs_SSP585_Compose.npy')

In [4]:
## 随机划分数据集
# 3个集合的年份数分配
Num_Year= 86
Num_Year_Train = 70
Num_Year_Validate = 8
Num_Year_Test = 8
# 设置随机数种子，使每次实验结果一致
random.seed(1)
Index_Year = list( range(Num_Year) )
Index_Year_Random = list( range(Num_Year) )
random.shuffle(Index_Year_Random) # 打乱年序号，后面根据年序号随机划分训练集、验证集和测试集
Index_Year_Train = Index_Year_Random[0:Num_Year_Train] # 训练集的序号
Index_Year_Validate = Index_Year_Random[Num_Year_Train:Num_Year_Train + Num_Year_Validate] # 验证集的序号
Index_Year_Test = Index_Year_Random[-Num_Year_Test::]  # 测试集的序号
## 数据集：变换成3维数组，即（组合数，年份数，12）
IMFs_ERSST_Compose_3D = IMFs_ERSST_Compose.reshape((6, Num_Year, 12)) 
IMFs_Historical_Compose_3D = IMFs_Historical_Compose.reshape((6, Num_Year, 12))
IMFs_SSP126_Compose_3D = IMFs_SSP126_Compose.reshape((6, Num_Year, 12))
IMFs_SSP245_Compose_3D = IMFs_SSP245_Compose.reshape((6, Num_Year, 12))
IMFs_SSP585_Compose_3D = IMFs_SSP585_Compose.reshape((6, Num_Year, 12))
# 对观测数据ERSST划分三个数据集合
IMFs_ERSST_Compose_3D_Train = IMFs_ERSST_Compose_3D[:,Index_Year_Train, :]
IMFs_ERSST_Compose_3D_Validate = IMFs_ERSST_Compose_3D[:,Index_Year_Validate, :]
IMFs_ERSST_Compose_3D_Test = IMFs_ERSST_Compose_3D[:,Index_Year_Test, :]
IMFs_ERSST_Compose_2D_Train = IMFs_ERSST_Compose_3D_Train.reshape((6,-1)) # 训练集 变成2维，方便计算每个组合的误差
IMFs_ERSST_Compose_2D_Validate = IMFs_ERSST_Compose_3D_Validate.reshape((6,-1)) # 验证集 变成2维，方便计算每个组合的误差
IMFs_ERSST_Compose_2D_Test = IMFs_ERSST_Compose_3D_Test.reshape((6,-1)) # 测试集 变成2维，方便计算每个组合的误差
# 模式的历史时期数据，同观测
IMFs_Historical_Compose_3D_Train = IMFs_Historical_Compose_3D[:,Index_Year_Train, :]
IMFs_Historical_Compose_3D_Validate = IMFs_Historical_Compose_3D[:,Index_Year_Validate, :]
IMFs_Historical_Compose_3D_Test = IMFs_Historical_Compose_3D[:,Index_Year_Test, :]
IMFs_Historical_Compose_2D_Train = IMFs_Historical_Compose_3D_Train.reshape((6,-1))
IMFs_Historical_Compose_2D_Validate = IMFs_Historical_Compose_3D_Validate.reshape((6,-1))
IMFs_Historical_Compose_2D_Test = IMFs_Historical_Compose_3D_Test.reshape((6,-1))

In [5]:
# 归一化：预分配变量空间
Input_Train_Scaler = np.zeros(IMFs_Historical_Compose_3D_Train.shape)
Output_Train_Scaler = np.zeros(IMFs_ERSST_Compose_3D_Train.shape)
Input_Validate_Scaler = np.zeros(IMFs_Historical_Compose_3D_Validate.shape)
Input_Test_Scaler = np.zeros(IMFs_Historical_Compose_3D_Test.shape)
Input_Forecast126_Scaler = np.zeros(IMFs_SSP126_Compose_3D.shape)
Input_Forecast245_Scaler = np.zeros(IMFs_SSP245_Compose_3D.shape)
Input_Forecast585_Scaler = np.zeros(IMFs_SSP585_Compose_3D.shape)
# 预设相关统计量和变量
RMSE_Predict_Train_All = np.zeros((6,9))  # (IMF的组合数, 待测试的隐层神经元数)
RMSE_Predict_Validate_All =  np.zeros((6,9))  # (IMF的组合数, 待测试的隐层神经元数)
RMSE_Predict_Test_All =  np.zeros((6,9))  # (IMF的组合数, 待测试的隐层神经元数)
Neuron_Predict_All = np.zeros(6)  # 储存每个组合验证集修正后误差最小时对应的隐层神经元数目
def rmse(predictions, targets):
    return np.sqrt( ( (predictions - targets)**2 ).mean() )
BP_Output_Train = np.zeros(IMFs_ERSST_Compose_3D_Train.shape)  # 反归一化后的原始值
BP_Output_Validate = np.zeros(IMFs_ERSST_Compose_3D_Validate.shape)
BP_Output_Test = np.zeros(IMFs_ERSST_Compose_3D_Test.shape)
BP_Output_SSP126 = np.zeros(IMFs_SSP126_Compose_3D.shape)
BP_Output_SSP245 = np.zeros(IMFs_SSP245_Compose_3D.shape)
BP_Output_SSP585 = np.zeros(IMFs_SSP585_Compose_3D.shape)
 
# 保存模型路径 
Path_Pre = '/home/mw/project/'
Path_SaveModel =  Path_Pre + 'Model/'
isExists = os.path.exists(Path_SaveModel)
if not isExists:
    os.makedirs(Path_Pre + 'Model/')
    print( Path_SaveModel +  ' successfully build!')
else:
    print( Path_SaveModel + ' has been existed!')

/home/mw/project/Model/ has been existed!


In [6]:
starttime = time.time()
for n in range(6):
    ## 归一化
    # 训练集
    Scaler_Input = MinMaxScaler(feature_range = (-1, 1) ) # 初始化输入的MinMaxScaler对象
    Input_Train_Scaler[n,:,:] = Scaler_Input.fit_transform(IMFs_Historical_Compose_3D_Train[n,:,:]) # 输入训练集归一化数组
    Scaler_Output = MinMaxScaler(feature_range = (-1, 1) ) # 初始化输出的MinMaxScaler对象
    Output_Train_Scaler[n,:,:] = Scaler_Output.fit_transform(IMFs_ERSST_Compose_3D_Train[n,:,:]) # 输出训练集归一化数组
    Input_Validate_Scaler[n,:,:] = Scaler_Input.transform(IMFs_Historical_Compose_3D_Validate[n,:,:])  # 依据训练集最大最小值，对输入验证集归一化
    Input_Test_Scaler[n,:,:] = Scaler_Input.transform(IMFs_Historical_Compose_3D_Test[n,:,:])  # 依据训练集最大最小值，对输入测试集归一化
    Input_Forecast126_Scaler[n,:,:] =  Scaler_Input.transform(IMFs_SSP126_Compose_3D[n,:,:]) # 依据训练集最大最小值，对输入未来预估低排放数据集归一化
    Input_Forecast245_Scaler[n,:,:] =  Scaler_Input.transform(IMFs_SSP245_Compose_3D[n,:,:]) # 依据训练集最大最小值，对输入未来预估中等排放数据集归一化
    Input_Forecast585_Scaler[n,:,:] =  Scaler_Input.transform(IMFs_SSP585_Compose_3D[n,:,:]) # 依据训练集最大最小值，对输入未来预估高排放数据集归一化

    ## 训练模型，尝试不同隐层神经元
    for number in range(6,15):  # 从6到15对隐藏层神经元调试
        # 建立模型
        Model_MLPR = MLPRegressor(solver = 'adam',  # 权重优化的求解器，选择的是随机梯度下降法
                                  activation = 'relu',  # 隐藏层的激活函数，选择relu函数
                                  alpha = 1e-4, # L2惩罚（正则项）参数
                                  learning_rate_init = 0.01, # 使用的初始学习率。
                                  max_iter = 200, # 最大迭代次数。
                                  hidden_layer_sizes = 3 , # 隐藏层神经元数目
                                  random_state = 1  ) # 决定用于权重和偏差初始化的随机数生成
        # 训练网络
        Model_MLPR.fit( Input_Train_Scaler[n,:,:], Output_Train_Scaler[n,:,:] )  # 输入: (样本条数, 输入层神经元数); 输出：(样本条数, 输出层神经元数)
        Predict_Train_Scaler = Model_MLPR.predict(Input_Train_Scaler[n, :, :])  # (训练样本年份数, 输入神经元数)
        # 反归一化
        Predict_Train_Real = Scaler_Output.inverse_transform( Predict_Train_Scaler )   # (训练样本年份数, 输入神经元数)
        Predict_Train_Real_1D = Predict_Train_Real.reshape(-1)   # (训练样本月份数, )
        # 计算训练RMSE
        RMSE_Train = rmse(IMFs_Historical_Compose_2D_Train[n, :], IMFs_ERSST_Compose_2D_Train[n,:] )
        RMSE_Predict_Train_All[n, number-6] = rmse(Predict_Train_Real_1D, IMFs_ERSST_Compose_2D_Train[n,:] )
        print('IMF number is: ', n + 1, ', neuron number is: ',number,  ', RMSE_Train: ', RMSE_Train, ',RMSE_Predict_Train: ', RMSE_Predict_Train_All[n, number-6])
       
        ## 验证集
        Predict_Validate_Scaler = Model_MLPR.predict(Input_Validate_Scaler[n, :, :])  # (验证样本年份数,输出层神经元数 )
        # 反归一化
        Predict_Validate_Real = Scaler_Output.inverse_transform( Predict_Validate_Scaler )   # (验证样本年份数, 输出层神经元数)
        Predict_Validate_Real_1D = Predict_Validate_Real.reshape(-1)  # (验证样本月份数, )
        # 计算验证集RMSE
        RMSE_Validate = rmse(IMFs_Historical_Compose_2D_Validate[n, :], IMFs_ERSST_Compose_2D_Validate[n,:] )
        RMSE_Predict_Validate_All[n, number - 6] = rmse(Predict_Validate_Real_1D, IMFs_ERSST_Compose_2D_Validate[n, :])
        print('IMF number is: ', n + 1, ', neuron number is: ',number, ', RMSE_Validate: ', RMSE_Validate, ',RMSE_Predict_Validate: ', RMSE_Predict_Validate_All[n, number-6])
        
        ## 测试集
        Predict_Test_Scaler = Model_MLPR.predict(Input_Test_Scaler[n, :, :])  # (测试样本年份数, 输出层神经元数)
        # 反归一化
        Predict_Test_Real = Scaler_Output.inverse_transform(Predict_Test_Scaler)  # (测试样本年份数, 输出层神经元数)
        Predict_Test_Real_1D = Predict_Test_Real.reshape(-1)  # (测试样本月份数, )
        # 计算测试集RMSE
        RMSE_Test = rmse(IMFs_Historical_Compose_2D_Test[n, :], IMFs_ERSST_Compose_2D_Test[n, :])
        RMSE_Predict_Test_All[n, number - 6] = rmse(Predict_Test_Real_1D, IMFs_ERSST_Compose_2D_Test[n, :])
        print('IMF number is: ', n + 1, ', neuron number is: ',number,  ', RMSE_Test: ', RMSE_Test, ',RMSE_Predict_Test: ', RMSE_Predict_Test_All[n, number - 6])
        
        ## SSP：未来预估订正
        Predict_SSP126_Scaler = Model_MLPR.predict(Input_Forecast126_Scaler[n, :, :])  # (未来预估年份数, 输出层神经元数)
        Predict_SSP245_Scaler = Model_MLPR.predict(Input_Forecast245_Scaler[n, :, :])  # (未来预估年份数, 输出层神经元数)
        Predict_SSP585_Scaler = Model_MLPR.predict(Input_Forecast585_Scaler[n, :, :])  # (未来预估年份数, 输出层神经元数)
        # 反归一化
        Predict_SSP126_Real = Scaler_Output.inverse_transform(Predict_SSP126_Scaler)  # (未来预估年份数, 输出层神经元数)
        Predict_SSP245_Real = Scaler_Output.inverse_transform(Predict_SSP245_Scaler)  # (未来预估年份数, 输出层神经元数)
        Predict_SSP585_Real = Scaler_Output.inverse_transform(Predict_SSP585_Scaler)  # (未来预估年份数, 输出层神经元数)

        ## 找到验证集最小的RMSE及其对应的隐层神经元数，以保存所有集合的订正结果
        if number == 6:
            RMSE_Predict_Validate_Min = rmse(Predict_Validate_Real_1D, IMFs_ERSST_Compose_2D_Validate[n, :] )
            print('--------------------------------------------------------------------------------------------------------------------------------')
            print('IMF number is: ', n + 1, ', neuron number is: ',number,  ', RMSE_Predict_Validate_Min: ', RMSE_Predict_Validate_Min)
            print('--------------------------------------------------------------------------------------------------------------------------------')
            Neuron_Predict_All[n] = number # 保存隐藏层神经元数，由于6是第一个测试数据，先当作误差最小时的保存下来，后面遇到误差更小的再把6替换
            BP_Output_Train[n,:,:] = Predict_Train_Real # 保存模型预测的就是订正的结果
            BP_Output_Validate[n,:,:] = Predict_Validate_Real # 保存验证集
            BP_Output_Test[n,:,:] = Predict_Test_Real # 保存测试集
            BP_Output_SSP126[n,:,:] = Predict_SSP126_Real # 保存未来预估的低排放情景ssp1-2.6
            BP_Output_SSP245[n,:,:] = Predict_SSP245_Real # 保存未来预估的中等排放情景ssp2-4.5
            BP_Output_SSP585[n,:,:] = Predict_SSP585_Real  # 保存未来预估的高排放情景ssp5-8.5
            exec('Model_MLPR_IMF_%s = Model_MLPR'%(n + 1) )
            exec("pickle.dump(Model_MLPR_IMF_%s, open(Path_SaveModel + 'Model_MLPR_IMF_%s', 'wb') )"%(n + 1, n + 1) )
        elif number >= 7: # 开始对大于6的隐层判断误差是否有比之前更小的
            RMSE_Predict_Validate_Current = rmse(Predict_Validate_Real_1D, IMFs_ERSST_Compose_2D_Validate[n, :] )
            if RMSE_Predict_Validate_Current < RMSE_Predict_Validate_Min: # 如果当前验证集误差比已经存在的最小误差更小，那就替换成最小误差
                RMSE_Predict_Validate_Min = RMSE_Predict_Validate_Current
                print('--------------------------------------------------------------------------------------------------------------------------------')
                print('IMF number is: ', n + 1, ', neuron number is: ', number, ', RMSE_Predict_Validate_Min: ', RMSE_Predict_Validate_Min)
                print('--------------------------------------------------------------------------------------------------------------------------------')
                Neuron_Predict_All[n] = number # 记录替换后的隐藏层神经元数目
                BP_Output_Train[n, :, :] = Predict_Train_Real
                BP_Output_Validate[n, :, :] = Predict_Validate_Real
                BP_Output_Test[n, :, :] = Predict_Test_Real
                BP_Output_SSP126[n, :, :] = Predict_SSP126_Real
                BP_Output_SSP245[n, :, :] = Predict_SSP245_Real
                BP_Output_SSP585[n, :, :] = Predict_SSP585_Real
                exec('Model_MLPR_IMF_%s = Model_MLPR' % (n + 1))
                exec("pickle.dump(Model_MLPR_IMF_%s, open(Path_SaveModel + 'Model_MLPR_IMF_%s', 'wb') )"%(n + 1, n + 1) )


endtime = time.time()
print('spend time is ', str(endtime - starttime) + "s")
print('Neuron_Predict_All :', Neuron_Predict_All)  # 打印输出各个组合最佳隐层数目

IMF number is:  1 , neuron number is:  6 , RMSE_Train:  0.2947259066099238 ,RMSE_Predict_Train:  0.02537035544629431
IMF number is:  1 , neuron number is:  6 , RMSE_Validate:  0.31819802043615875 ,RMSE_Predict_Validate:  0.03379236897362349
IMF number is:  1 , neuron number is:  6 , RMSE_Test:  0.23910681519244498 ,RMSE_Predict_Test:  0.024146507307126357
--------------------------------------------------------------------------------------------------------------------------------
IMF number is:  1 , neuron number is:  6 , RMSE_Predict_Validate_Min:  0.03379236897362349
--------------------------------------------------------------------------------------------------------------------------------
IMF number is:  1 , neuron number is:  7 , RMSE_Train:  0.2947259066099238 ,RMSE_Predict_Train:  0.02537035544629431
IMF number is:  1 , neuron number is:  7 , RMSE_Validate:  0.31819802043615875 ,RMSE_Predict_Validate:  0.03379236897362349
IMF number is:  1 , neuron number is:  7 , RMSE_Test

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


IMF number is:  5 , neuron number is:  6 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  6 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  6 , RMSE_Test:  0.05183950143043771 ,RMSE_Predict_Test:  0.03736406905560681
--------------------------------------------------------------------------------------------------------------------------------
IMF number is:  5 , neuron number is:  6 , RMSE_Predict_Validate_Min:  0.03192954735868749
--------------------------------------------------------------------------------------------------------------------------------
IMF number is:  5 , neuron number is:  7 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  7 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  7 , RMSE_T

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


IMF number is:  5 , neuron number is:  9 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  9 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  9 , RMSE_Test:  0.05183950143043771 ,RMSE_Predict_Test:  0.03736406905560681
IMF number is:  5 , neuron number is:  10 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  10 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  10 , RMSE_Test:  0.05183950143043771 ,RMSE_Predict_Test:  0.03736406905560681
IMF number is:  5 , neuron number is:  11 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  11 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number 

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


IMF number is:  5 , neuron number is:  12 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  12 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  12 , RMSE_Test:  0.05183950143043771 ,RMSE_Predict_Test:  0.03736406905560681
IMF number is:  5 , neuron number is:  13 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  13 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron number is:  13 , RMSE_Test:  0.05183950143043771 ,RMSE_Predict_Test:  0.03736406905560681
IMF number is:  5 , neuron number is:  14 , RMSE_Train:  0.061813553491955116 ,RMSE_Predict_Train:  0.02770939813583323
IMF number is:  5 , neuron number is:  14 , RMSE_Validate:  0.05405021608910473 ,RMSE_Predict_Validate:  0.03192954735868749
IMF number is:  5 , neuron numb

In [7]:
# 按时间顺序重新排列修正数据
BP_Output_All = np.zeros(IMFs_ERSST_Compose_3D.shape)
BP_Output_All[:, 0:Num_Year_Train, :] = BP_Output_Train
BP_Output_All[:, Num_Year_Train:Num_Year_Train +
              Num_Year_Validate, :] = BP_Output_Validate
BP_Output_All[:, -Num_Year_Test::, :] = BP_Output_Test

BP_Output_All_Chronological = np.zeros(IMFs_ERSST_Compose_3D.shape)
for n in range(Num_Year):
    Index_Year_Real = Index_Year_Random[n]
    BP_Output_All_Chronological[:, Index_Year_Real, :] = BP_Output_All[:, n, :]

# 保存数据
Path_Pre = '/home/mw/project/SaveData/'
Path_SaveModelData = Path_Pre + 'Model/'
isExists = os.path.exists(Path_SaveModelData)
if not isExists:
    os.makedirs(Path_Pre + 'Model/')
    print(Path_SaveModelData + ' successfully build!')
else:
    print(Path_SaveModelData + ' has been existed!')

np.save(
    Path_SaveModelData +
    'BP_Output_All_Chronological.npy',
    BP_Output_All_Chronological)
np.save(Path_SaveModelData + 'BP_Output_SSP126.npy', BP_Output_SSP126)
np.save(Path_SaveModelData + 'BP_Output_SSP245.npy', BP_Output_SSP245)
np.save(Path_SaveModelData + 'BP_Output_SSP585.npy', BP_Output_SSP585)

/home/mw/project/SaveData/Model/ has been existed!


In [8]:
#BPNN订正结果分析
# 原始数据
Path_SaveData_Original = '/home/mw/project/SaveData/Original/'
SST_ERSST_AreaMean = np.load(Path_SaveData_Original + 'SST_ERSST_AreaMean.npy')
SST_Historical_AreaMean = np.load(
    Path_SaveData_Original +
    'SST_Historical_AreaMean.npy')
SST_SSP126 = np.load(Path_SaveData_Original + 'SST_SSP126_AreaMean.npy')
SST_SSP245 = np.load(Path_SaveData_Original + 'SST_SSP245_AreaMean.npy')
SST_SSP585 = np.load(Path_SaveData_Original + 'SST_SSP585_AreaMean.npy')
Time_ERSST = np.load(Path_SaveData_Original + 'Time_ERSST.npy')
Time_SSP = np.load(Path_SaveData_Original + 'Time_SSP.npy')
# 数据集划分
Num_Year = 86
Num_Year_Train = 70
Num_Year_Validate = 8
Num_Year_Test = 8
Num_Sample_SST = Num_Year*12
Num_Sample_Train = Num_Year_Train*12
Num_Sample_Validate = Num_Year_Validate*12
Num_Sample_Test = Num_Year_Test*12
Time_SST = np.arange(0, Num_Sample_SST)
Time_ERSST = Time_ERSST[-Num_Sample_SST::]
SST_ERSST = SST_ERSST_AreaMean[-Num_Sample_SST::]
SST_Historical = SST_Historical_AreaMean[-Num_Sample_SST::]
SST_ERSST_2D = SST_ERSST.reshape((Num_Year, 12))
SST_Historical_2D = SST_Historical.reshape((Num_Year, 12))
# 随机数
random.seed(1)
Index_Year = list(range(Num_Year))
Index_Year_Random = list(range(Num_Year))
random.shuffle(Index_Year_Random)
Index_Year_Train = Index_Year_Random[0:Num_Year_Train]
Index_Year_Validate = Index_Year_Random[Num_Year_Train:Num_Year_Train +
                                        Num_Year_Validate]
Index_Year_Test = Index_Year_Random[-Num_Year_Test::]
# 原始数据：2维（年份数，12）
SST_ERSST_Train_2D = SST_ERSST_2D[Index_Year_Train, :]
SST_ERSST_Validate_2D = SST_ERSST_2D[Index_Year_Validate, :]
SST_ERSST_Test_2D = SST_ERSST_2D[Index_Year_Test, :]
SST_Historical_Train_2D = SST_Historical_2D[Index_Year_Train, :]
SST_Historical_Validate_2D = SST_Historical_2D[Index_Year_Validate, :]
SST_Historical_Test_2D = SST_Historical_2D[Index_Year_Test, :]
# 原始数据：1维（月份数，）
SST_ERSST_All_1D = SST_ERSST_2D.reshape(-1)
SST_ERSST_Train_1D = SST_ERSST_Train_2D.reshape(-1)
SST_ERSST_Validate_1D = SST_ERSST_Validate_2D.reshape(-1)
SST_ERSST_Test_1D = SST_ERSST_Test_2D.reshape(-1)
SST_Historical_All_1D = SST_Historical_2D.reshape(-1)
SST_Historical_Train_1D = SST_Historical_Train_2D.reshape(-1)
SST_Historical_Validate_1D = SST_Historical_Validate_2D.reshape(-1)
SST_Historical_Test_1D = SST_Historical_Test_2D.reshape(-1)

In [9]:
#EEMD分解的数据
Path_SaveData_EEMD = '/home/mw/project/SaveData/EEMD/'
IMFs_ERSST_Compose = np.load(Path_SaveData_EEMD + 'IMFs_ERSST_Compose.npy')
IMFs_Historical_Compose = np.load(
    Path_SaveData_EEMD +
    'IMFs_Historical_Compose.npy')
IMFs_SSP126_Compose = np.load(Path_SaveData_EEMD + 'IMFs_SSP126_Compose.npy')
IMFs_SSP245_Compose = np.load(Path_SaveData_EEMD + 'IMFs_SSP245_Compose.npy')
IMFs_SSP585_Compose = np.load(Path_SaveData_EEMD + 'IMFs_SSP585_Compose.npy')
# IMF: 2维 (6, 月份数)
IMFs_ERSST_Compose_3D = IMFs_ERSST_Compose.reshape((6, Num_Year, 12))
IMFs_ERSST_Compose_All_2D = IMFs_ERSST_Compose_3D.reshape((6, -1))
IMFs_ERSST_Compose_Train_2D = IMFs_ERSST_Compose_3D[:, Index_Year_Train, :].reshape(
    (6, -1))
IMFs_ERSST_Compose_Validate_2D = IMFs_ERSST_Compose_3D[:, Index_Year_Validate, :].reshape(
    (6, -1))
IMFs_ERSST_Compose_Test_2D = IMFs_ERSST_Compose_3D[:, Index_Year_Test, :].reshape(
    (6, -1))
IMFs_Historical_Compose_3D = IMFs_Historical_Compose.reshape((6, Num_Year, 12))
IMFs_Historical_Compose_All_2D = IMFs_Historical_Compose_3D.reshape((6, -1))
IMFs_Historical_Compose_Train_2D = IMFs_Historical_Compose_3D[:, Index_Year_Train, :].reshape(
    (6, -1))
IMFs_Historical_Compose_Validate_2D = IMFs_Historical_Compose_3D[:, Index_Year_Validate, :].reshape(
    (6, -1))
IMFs_Historical_Compose_Test_2D = IMFs_Historical_Compose_3D[:, Index_Year_Test, :].reshape(
    (6, -1))

In [10]:
#BPNN输出数据
Path_SaveData_Model = '/home/mw/project/SaveData/Model/'
IMFs_Historical_Compose_Correct_3D = np.load(
    Path_SaveData_Model + 'BP_Output_All_Chronological.npy')
IMFs_SSP126_Compose_Correct_3D = np.load(
    Path_SaveData_Model + 'BP_Output_SSP126.npy')
IMFs_SSP245_Compose_Correct_3D = np.load(
    Path_SaveData_Model + 'BP_Output_SSP245.npy')
IMFs_SSP585_Compose_Correct_3D = np.load(
    Path_SaveData_Model + 'BP_Output_SSP585.npy')
# 订正后IMF: 2维 (6,年份数)
IMFs_Historical_Compose_Correct_All_2D = IMFs_Historical_Compose_Correct_3D.reshape(
    (6, -1))
IMFs_Historical_Compose_Correct_Train_2D = IMFs_Historical_Compose_Correct_3D[:, Index_Year_Train, :].reshape(
    (6, -1))
IMFs_Historical_Compose_Correct_Validate_2D = IMFs_Historical_Compose_Correct_3D[:, Index_Year_Validate, :].reshape(
    (6, -1))
IMFs_Historical_Compose_Correct_Test_2D = IMFs_Historical_Compose_Correct_3D[:, Index_Year_Test, :].reshape(
    (6, -1))
# 重构的订正IMF: 1维 (月份数,)
SST_Historical_Correct_All_1D = np.sum(
    IMFs_Historical_Compose_Correct_All_2D, 0)
SST_Historical_Correct_Train_1D = np.sum(
    IMFs_Historical_Compose_Correct_Train_2D, 0)
SST_Historical_Correct_Validate_1D = np.sum(
    IMFs_Historical_Compose_Correct_Validate_2D, 0)
SST_Historical_Correct_Test_1D = np.sum(
    IMFs_Historical_Compose_Correct_Test_2D, 0)
SST_SSP126_Correct_1D = np.sum(
    IMFs_SSP126_Compose_Correct_3D.reshape(
        (6, -1)), 0)
SST_SSP245_Correct_1D = np.sum(
    IMFs_SSP245_Compose_Correct_3D.reshape(
        (6, -1)), 0)
SST_SSP585_Correct_1D = np.sum(
    IMFs_SSP585_Compose_Correct_3D.reshape(
        (6, -1)), 0)
# 重构的订正IMF: 2维 (年份数, 12)
SST_Historical_Correct_All_2D = SST_Historical_Correct_All_1D.reshape((-1, 12))
SST_Historical_Correct_Train_2D = SST_Historical_Correct_Train_1D.reshape(
    (-1, 12))
SST_Historical_Correct_Validate_2D = SST_Historical_Correct_Validate_1D.reshape(
    (-1, 12))
SST_Historical_Correct_Test_2D = SST_Historical_Correct_Test_1D.reshape(
    (-1, 12))

In [11]:
#结果分析
### 计算SST IMF的： 均方根误差（RMSE）/相关系数（R）
## 均方根误差
RMSE_IMFs_Historical_Compose_Train = np.zeros(6)
RMSE_IMFs_Historical_Compose_Validate = np.zeros(6)
RMSE_IMFs_Historical_Compose_Test = np.zeros(6)
RMSE_IMFs_Historical_Compose_All = np.zeros(6)
RMSE_IMFs_Historical_Compose_Correct_Train = np.zeros(6)
RMSE_IMFs_Historical_Compose_Correct_Validate = np.zeros(6)
RMSE_IMFs_Historical_Compose_Correct_Test = np.zeros(6)
RMSE_IMFs_Historical_Compose_Correct_All = np.zeros(6)
for n in range(6):
    # 原始
    RMSE_IMFs_Historical_Compose_Train[n] = rmse( IMFs_Historical_Compose_Train_2D[n,:], IMFs_ERSST_Compose_Train_2D[n,:] )
    RMSE_IMFs_Historical_Compose_Validate[n] = rmse( IMFs_Historical_Compose_Validate_2D[n,:], IMFs_ERSST_Compose_Validate_2D[n,:] )
    RMSE_IMFs_Historical_Compose_Test[n] = rmse( IMFs_Historical_Compose_Test_2D[n,:], IMFs_ERSST_Compose_Test_2D[n,:] )
    RMSE_IMFs_Historical_Compose_All[n] = rmse( IMFs_Historical_Compose_All_2D[n,:], IMFs_ERSST_Compose_All_2D[n,:] )
    # 订正
    RMSE_IMFs_Historical_Compose_Correct_Train[n] = rmse( IMFs_Historical_Compose_Correct_Train_2D[n,:], IMFs_ERSST_Compose_Train_2D[n,:] )
    RMSE_IMFs_Historical_Compose_Correct_Validate[n] = rmse( IMFs_Historical_Compose_Correct_Validate_2D[n,:], IMFs_ERSST_Compose_Validate_2D[n,:] )
    RMSE_IMFs_Historical_Compose_Correct_Test[n] = rmse( IMFs_Historical_Compose_Correct_Test_2D[n,:], IMFs_ERSST_Compose_Test_2D[n,:] )
    RMSE_IMFs_Historical_Compose_Correct_All[n] = rmse( IMFs_Historical_Compose_Correct_All_2D[n,:], IMFs_ERSST_Compose_All_2D[n,:] )

## 相关系数
R_IMFs_Historical_Compose_Train = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Validate = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Test = np.zeros( (6,2) )
R_IMFs_Historical_Compose_All = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Correct_Train = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Correct_Validate = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Correct_Test = np.zeros( (6,2) )
R_IMFs_Historical_Compose_Correct_All = np.zeros( (6,2) )
for n in range(6):
    # 原始
    R_IMFs_Historical_Compose_Train[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Train_2D[n,:], IMFs_ERSST_Compose_Train_2D[n,:] )[:]
    R_IMFs_Historical_Compose_Validate[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Validate_2D[n,:], IMFs_ERSST_Compose_Validate_2D[n,:] )[:]
    R_IMFs_Historical_Compose_Test[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Test_2D[n,:], IMFs_ERSST_Compose_Test_2D[n,:] )[:]
    R_IMFs_Historical_Compose_All[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_All_2D[n,:], IMFs_ERSST_Compose_All_2D[n,:] )[:]
    # 订正
    R_IMFs_Historical_Compose_Correct_Train[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Correct_Train_2D[n,:], IMFs_ERSST_Compose_Train_2D[n,:] )[:]
    R_IMFs_Historical_Compose_Correct_Validate[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Correct_Validate_2D[n,:], IMFs_ERSST_Compose_Validate_2D[n,:] )[:]
    R_IMFs_Historical_Compose_Correct_Test[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Correct_Test_2D[n,:], IMFs_ERSST_Compose_Test_2D[n,:] )[:]
    R_IMFs_Historical_Compose_Correct_All[n,:] = scipy.stats.pearsonr( IMFs_Historical_Compose_Correct_All_2D[n,:], IMFs_ERSST_Compose_All_2D[n,:] )[:]
## 结果较多，大家可打印自行查看

In [12]:
### 计算重构SST的均方根误差/相关系数
## 均方根误差
RMSE_SST_Historical_Train = rmse( SST_Historical_Train_1D, SST_ERSST_Train_1D )
RMSE_SST_Historical_Validate = rmse( SST_Historical_Validate_1D, SST_ERSST_Validate_1D )
RMSE_SST_Historical_Test = rmse( SST_Historical_Test_1D, SST_ERSST_Test_1D )
RMSE_SST_Historical_All = rmse( SST_Historical_All_1D, SST_ERSST_All_1D )
RMSE_SST_Historical_Correct_Train = rmse( SST_Historical_Correct_Train_1D, SST_ERSST_Train_1D )
RMSE_SST_Historical_Correct_Validate = rmse( SST_Historical_Correct_Validate_1D, SST_ERSST_Validate_1D )
RMSE_SST_Historical_Correct_Test = rmse( SST_Historical_Correct_Test_1D, SST_ERSST_Test_1D )
RMSE_SST_Historical_Correct_All = rmse( SST_Historical_Correct_All_1D, SST_ERSST_All_1D )
# 打印查看订正前后均方根误差
print(RMSE_SST_Historical_Train, RMSE_SST_Historical_Validate, RMSE_SST_Historical_Test, RMSE_SST_Historical_All)
print(RMSE_SST_Historical_Correct_Train, RMSE_SST_Historical_Correct_Validate, RMSE_SST_Historical_Correct_Test, RMSE_SST_Historical_Correct_All)

## 相关系数
R_SST_Historical_Train = scipy.stats.pearsonr( SST_Historical_Train_1D, SST_ERSST_Train_1D )[:]
R_SST_Historical_Validate = scipy.stats.pearsonr( SST_Historical_Validate_1D, SST_ERSST_Validate_1D )[:]
R_SST_Historical_Test = scipy.stats.pearsonr( SST_Historical_Test_1D, SST_ERSST_Test_1D )[:]
R_SST_Historical_All = scipy.stats.pearsonr( SST_Historical_All_1D, SST_ERSST_All_1D )[:]
R_SST_Historical_Correct_Train = scipy.stats.pearsonr( SST_Historical_Correct_Train_1D, SST_ERSST_Train_1D )[:]
R_SST_Historical_Correct_Validate = scipy.stats.pearsonr( SST_Historical_Correct_Validate_1D, SST_ERSST_Validate_1D )[:]
R_SST_Historical_Correct_Test = scipy.stats.pearsonr( SST_Historical_Correct_Test_1D, SST_ERSST_Test_1D )[:]
R_SST_Historical_Correct_All = scipy.stats.pearsonr( SST_Historical_Correct_All_1D, SST_ERSST_All_1D )[:]
# 打印查看订正前后相关系数
print(R_SST_Historical_Train, R_SST_Historical_Validate, R_SST_Historical_Test, R_SST_Historical_All)
print(R_SST_Historical_Correct_Train, R_SST_Historical_Correct_Validate, R_SST_Historical_Correct_Test, R_SST_Historical_Correct_All)

2.149957341101214 2.1572594459521266 2.0624753304214547 2.142653056172619
0.10473225135838651 0.13860303153374104 0.11105692843073407 0.10891508570229214
(0.12629002119270538, 0.00024304106108895051) (0.11713976128267511, 0.2556996997241975) (0.11610335077149972, 0.25995872529037556) (0.12541039222217268, 5.3484242704139465e-05)
(0.9096220052552149, 6.13e-322) (0.818962605023977, 2.1163617116423683e-24) (0.8467875699239354, 1.64094265359744e-27) (0.8964850984400696, 0.0)


In [13]:
### 画重构SST
## 预设
fontsizenum = 16  
markersizenum = 6
Num_Year = 2014-1929+1
Num_Sample_SST = Num_Year*12
X_Time = np.arange(0, Num_Sample_SST )
Time_SST = np.array(Time_ERSST[-Num_Sample_SST::])
xtick_SST_interval = 12*12
xticklabel_SST = list(range(0, Num_Sample_SST, xtick_SST_interval)); print(xticklabel_SST)
xticklabels_SST = Time_SST[xticklabel_SST[0::]].astype('str').tolist()
ylim_sst_max = 19
ylim_sst_min = 17
delta_ylim_sst = ylim_sst_max - ylim_sst_min
yticklabel_SST = np.linspace(ylim_sst_min, ylim_sst_max, num = 5).tolist()
yticklabel_SST = [round(i,1) for i in yticklabel_SST]; print(yticklabel_SST)
yticklabels_SST = [str(i) for i in yticklabel_SST]; print(yticklabels_SST)
## 画
fig, ax = plt.subplots(1, 1, figsize = (16,9), dpi = 600)
ax.plot(X_Time, SST_Historical_All_1D, 'b-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'blue', label = 'FIO-ESM v2')
ax.plot(X_Time, SST_Historical_Correct_All_1D, 'r-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'red', label = 'EEMD-BPNN')
ax.plot(X_Time, SST_ERSST_All_1D, 'k-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'black', label = 'ERSST')
ax.set_xlim(0, Num_Sample_SST)
ax.set_ylim(ylim_sst_min, ylim_sst_max)  
ax.set_xticks(xticklabel_SST)
ax.set_yticks(yticklabel_SST)  
ax.set_xticklabels(xticklabels_SST, fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_SST, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize=fontsizenum)
ax.set_ylabel('SST(℃)', fontsize = fontsizenum)
legend_font = { 'weight': 'normal', 'size': fontsizenum}
ax.legend(loc = 'upper left', frameon = False, prop = legend_font, ncol = 1)
ax.grid(linestyle='-.')
## 保存图片
Path_Pre = '/home/mw/project/Figures/'
Path_SaveFigures =  Path_Pre + 'Model/'
isExists = os.path.exists(Path_SaveFigures)
if not isExists:
    os.makedirs(Path_Pre + 'Model/')
    print( Path_SaveFigures +  ' successfully build!')
else:
    print( Path_SaveFigures + ' has been existed!')

fns = os.path.join(Path_SaveFigures, 'SST_Historical_Correct_ERSST.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight')
plt.show()
plt.close()

[0, 144, 288, 432, 576, 720, 864, 1008]
[17.0, 17.5, 18.0, 18.5, 19.0]
['17.0', '17.5', '18.0', '18.5', '19.0']
/home/mw/project/Figures/Model/ has been existed!


<Figure size 9600x5400 with 1 Axes>

In [14]:
Bias_Historical_Train_1D = SST_Historical_Train_1D - SST_ERSST_Train_1D
Bias_Historical_Validate_1D = SST_Historical_Validate_1D - SST_ERSST_Validate_1D
Bias_Historical_Test_1D = SST_Historical_Test_1D - SST_ERSST_Test_1D
Bias_Historical_All_1D = SST_Historical_All_1D - SST_ERSST_All_1D
Bias_Historical_Correct_Train_1D = SST_Historical_Correct_Train_1D - SST_ERSST_Train_1D
Bias_Historical_Correct_Validate_1D = SST_Historical_Correct_Validate_1D - SST_ERSST_Validate_1D
Bias_Historical_Correct_Test_1D = SST_Historical_Correct_Test_1D - SST_ERSST_Test_1D
Bias_Historical_Correct_All_1D = SST_Historical_Correct_All_1D - SST_ERSST_All_1D
### 计算重构的SST偏差: 最小值（Min）/最大值（Max）/平均偏差（Mean Bias）/标准差（Std），这些指标大家可以打印查看
## 最小值
Min_Bias_Historical_Train_1D = np.min(Bias_Historical_Train_1D)
Min_Bias_Historical_Validate_1D = np.min(Bias_Historical_Validate_1D)
Min_Bias_Historical_Test_1D = np.min(Bias_Historical_Test_1D)
Min_Bias_Historical_All_1D = np.min(Bias_Historical_All_1D)
Min_Bias_Historical_Correct_Train_1D = np.min(Bias_Historical_Correct_Train_1D)
Min_Bias_Historical_Correct_Validate_1D = np.min(Bias_Historical_Validate_1D)
Min_Bias_Historical_Correct_Test_1D = np.min(Bias_Historical_Correct_Test_1D)
Min_Bias_Historical_Correct_All_1D = np.min(Bias_Historical_Correct_All_1D)
## 最大值
Max_Bias_Historical_Train_1D = np.max(Bias_Historical_Train_1D)
Max_Bias_Historical_Validate_1D = np.max(Bias_Historical_Validate_1D)
Max_Bias_Historical_Test_1D = np.max(Bias_Historical_Test_1D)
Max_Bias_Historical_All_1D = np.max(Bias_Historical_All_1D)
Max_Bias_Historical_Correct_Train_1D = np.max(Bias_Historical_Correct_Train_1D)
Max_Bias_Historical_Correct_Validate_1D = np.max(Bias_Historical_Validate_1D)
Max_Bias_Historical_Correct_Test_1D = np.max(Bias_Historical_Correct_Test_1D)
Max_Bias_Historical_Correct_All_1D = np.max(Bias_Historical_Correct_All_1D)
# 平均偏差
Mean_Bias_Historical_Train_1D = np.mean(Bias_Historical_Train_1D)
Mean_Bias_Historical_Validate_1D = np.mean(Bias_Historical_Validate_1D)
Mean_Bias_Historical_Test_1D = np.mean(Bias_Historical_Test_1D)
Mean_Bias_Historical_All_1D = np.mean(Bias_Historical_All_1D)
Mean_Bias_Historical_Correct_Train_1D = np.mean(Bias_Historical_Correct_Train_1D)
Mean_Bias_Historical_Correct_Validate_1D = np.mean(Bias_Historical_Validate_1D)
Mean_Bias_Historical_Correct_Test_1D = np.mean(Bias_Historical_Correct_Test_1D)
Mean_Bias_Historical_Correct_All_1D = np.mean(Bias_Historical_Correct_All_1D)
# 标准差
Std_Bias_Historical_Train_1D = np.std(Bias_Historical_Train_1D, ddof = 0)
Std_Bias_Historical_Validate_1D = np.std(Bias_Historical_Validate_1D, ddof = 0)
Std_Bias_Historical_Test_1D = np.std(Bias_Historical_Test_1D, ddof = 0)
Std_Bias_Historical_All_1D = np.std(Bias_Historical_All_1D, ddof = 0)
Std_Bias_Historical_Correct_Train_1D = np.std(Bias_Historical_Correct_Train_1D, ddof = 0)
Std_Bias_Historical_Correct_Validate_1D = np.std(Bias_Historical_Validate_1D, ddof = 0)
Std_Bias_Historical_Correct_Test_1D = np.std(Bias_Historical_Correct_Test_1D, ddof = 0)
Std_Bias_Historical_Correct_All_1D = np.std(Bias_Historical_Correct_All_1D, ddof = 0)

###  画重构的SST偏差
## 预设
fontsizenum = 16 
markersizenum = 6
Num_Year = 2014-1929+1
Num_Sample_Bias = Num_Year*12
X_Time = np.arange(0, Num_Sample_Bias )
Time_Bias = np.array(Time_ERSST[-Num_Sample_Bias::])
xtick_Bias_interval = 12*12
xticklabel_Bias = list(range(0, Num_Sample_Bias, xtick_Bias_interval)); print(xticklabel_Bias)
xticklabels_Bias = Time_Bias[xticklabel_Bias[0::]].astype('str').tolist()
ylim_bias_max = 0.6
ylim_bias_min = -0.6
delta_ylim_bias = ylim_bias_max - ylim_bias_min
yticklabel_Bias = np.linspace(ylim_bias_min, ylim_bias_max, num = 7).tolist()
yticklabel_Bias = [round(i,1) for i in yticklabel_Bias]; print(yticklabel_Bias)
yticklabels_Bias = [str(i) for i in yticklabel_Bias]; print(yticklabels_Bias)
### 画
fig, ax = plt.subplots(1, 1, figsize = (16,9), dpi = 600)
ax.plot(X_Time, Bias_Historical_All_1D, 'b-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'blue', label = 'FIO-ESM v2')
ax.plot(X_Time, Bias_Historical_Correct_All_1D, 'r-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'red', label = 'EEMD-BPNN')
plt.axhline(y = 0 , c = 'k', ls = '--', lw = 3)
ax.set_xlim(0, Num_Sample_Bias)
ax.set_ylim(ylim_bias_min, ylim_bias_max)  
ax.set_xticks(xticklabel_Bias)
ax.set_yticks(yticklabel_Bias)  
ax.set_xticklabels(xticklabels_Bias, fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_Bias, fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize=fontsizenum)
ax.set_ylabel('SST Bias(℃)', fontsize = fontsizenum)
legend_font = { 'weight': 'normal', 'size': fontsizenum}
ax.legend(loc = 'upper left', frameon = False, prop = legend_font, ncol = 1)
ax.grid(linestyle='-.')
## 保存图片
fns = os.path.join(Path_SaveFigures, 'SSTBias_Historical_Correct.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight')
plt.show()
plt.close()

[0, 144, 288, 432, 576, 720, 864, 1008]
[-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6]
['-0.6', '-0.4', '-0.2', '0.0', '0.2', '0.4', '0.6']


<Figure size 9600x5400 with 1 Axes>

In [15]:
### 均方根误差
RMSE_SST_Historical_12Month_All = np.zeros(12)
RMSE_SST_Historical_12Month_Correct_All = np.zeros(12)
for month in range(12):
    RMSE_SST_Historical_12Month_All[month] = rmse(SST_Historical_2D[:,month], SST_ERSST_2D[:,month])
    RMSE_SST_Historical_12Month_Correct_All[month] = rmse(SST_Historical_Correct_All_2D[:,month], SST_ERSST_2D[:,month])
print(RMSE_SST_Historical_12Month_All)
print(RMSE_SST_Historical_12Month_Correct_All)
### 相关系数
R_SST_Historical_12Month_All = np.zeros( (12,2) )
R_SST_Historical_12Month_Correct_All = np.zeros( (12,2) )
for month in range(12):
    R_SST_Historical_12Month_All[month,:] = scipy.stats.pearsonr(SST_Historical_2D[:,month], SST_ERSST_2D[:,month])[:]
    R_SST_Historical_12Month_Correct_All[month,:] = scipy.stats.pearsonr(SST_Historical_Correct_All_2D[:,month], SST_ERSST_2D[:,month])[:]
print(R_SST_Historical_12Month_All)
print(R_SST_Historical_12Month_Correct_All)

### 画图：重构SST的逐月表现（均方根误差/相关系数）
## 预设
str_panels = ['a)RMSE', 'b)R']
fontsizenum = 16  
barwidthnum = 0.3
X_Month = np.arange(1,13)
xticklabel_Month = list(range(0,14))
xticklabels_Month = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','']
## 均方根误差
ylim_rmse_max = 0.4
ylim_rmse_min = 0
delta_rmse_bias = ylim_rmse_max - ylim_rmse_min
yticklabel_RMSE = np.linspace(ylim_rmse_min, ylim_rmse_max, num = 9).tolist()
yticklabel_RMSE = [round(i,2) for i in yticklabel_RMSE]; print(yticklabel_RMSE)
yticklabels_RMSE = [str(i) for i in yticklabel_RMSE]; print(yticklabels_RMSE)
## 相关系数
ylim_R_max = 1.0
ylim_R_min = 0.7
delta_R_bias = ylim_R_max - ylim_R_min
yticklabel_R = np.linspace(ylim_R_min, ylim_R_max, num = 7).tolist()
yticklabel_R = [round(i,2) for i in yticklabel_R]; print(yticklabel_R)
yticklabels_R = [str(i) for i in yticklabel_R]; print(yticklabels_R)

## 画图
fig, ax = plt.subplots(2, 1, figsize = (16,9), dpi = 600)
# 均方根误差
ax = plt.subplot(2, 1, 1)
bar1 = ax.bar(X_Month - barwidthnum, RMSE_SST_Historical_12Month_All, color = 'blue',  alpha = 1,  width = barwidthnum, edgecolor = 'black', linewidth = 1.5, bottom = 0, align = 'edge') # label = 'FIO-ESM v2'
bar2 = ax.bar(X_Month, RMSE_SST_Historical_12Month_Correct_All, color = 'red',  alpha = 1,  width = barwidthnum, edgecolor = 'black', linewidth = 1.5, bottom = 0, align = 'edge') # label = 'EEMD-BPNN'
for month in range(12):
    Y_Month = np.round( RMSE_SST_Historical_12Month_All[month], 3)
    Y_Month_Correct = np.round( RMSE_SST_Historical_12Month_Correct_All[month], 3)
    Str_Y_Month = str(Y_Month)
    Str_Y_Month_Correct = str(Y_Month_Correct)
    ax.text(X_Month[month] - barwidthnum/2, Y_Month + 0.01, Str_Y_Month, horizontalalignment ='center', fontsize = 10 )
    ax.text(X_Month[month] + barwidthnum/2, Y_Month_Correct + 0.01, Str_Y_Month_Correct, horizontalalignment ='center', fontsize = 10 )
plot1 = plt.axhline(y = RMSE_SST_Historical_All , c = 'blue', ls = '-', lw = 2)
plot2 = plt.axhline(y = RMSE_SST_Historical_Correct_All , c = 'red', ls = '-', lw = 2)
Y_Month = np.round(RMSE_SST_Historical_All, 3); Str_Y_Month = str(Y_Month)
Y_Month_Correct = np.round(RMSE_SST_Historical_Correct_All, 3); Str_Y_Month_Correct = str(Y_Month_Correct)
ax.text(0.3, Y_Month + 0.01, Str_Y_Month, horizontalalignment='center', fontsize = 14)
ax.text(0.3, Y_Month_Correct + 0.01, Str_Y_Month_Correct, horizontalalignment='center', fontsize = 14)
ax.text(0, ylim_rmse_min + delta_rmse_bias*0.92, str_panels[0], fontsize=fontsizenum)   # color = 'black'
ax.set(xlim = (0, 13), ylim = (ylim_rmse_min, ylim_rmse_max))
ax.tick_params(axis = 'both', which = 'both', direction = 'in')
ax.set_xticks(xticklabel_Month)
ax.set_yticks(yticklabel_RMSE)
ax.set_xticklabels(xticklabels_Month, fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_RMSE, fontsize = fontsizenum)
ax.set_xlabel('Month', fontsize=fontsizenum)
ax.set_ylabel('RMSE(℃)', fontsize = fontsizenum)
ax.grid(axis = 'y', linestyle = '--', lw = 1)  # color='grey', alpha=0.5
legend_font = {'weight': 'normal', 'size': fontsizenum}
plt.legend([bar1, bar2, plot1, plot2], ["FIO-ESM v2", "EEMD-BPNN", "FIO-ESM v2(All Time)","EEMD-BPNN(All Time)"],
           bbox_to_anchor = (0.3, 1.0), loc = 'upper center', ncol = 2, frameon = False, prop = legend_font)

# 相关系数
ax = plt.subplot(2, 1, 2)
bar1 = ax.bar(X_Month - barwidthnum, R_SST_Historical_12Month_All[:,0], color = 'blue',  alpha = 1,  width = barwidthnum, edgecolor = 'black', linewidth = 1.5, bottom = 0, align = 'edge') # label = 'FIO-EM v2'
bar2 = ax.bar(X_Month, R_SST_Historical_12Month_Correct_All[:,0], color = 'red',  alpha = 1,  width = barwidthnum, edgecolor = 'black', linewidth = 1.5, bottom = 0, align = 'edge') # label = 'FIO-EM v2'
for month in range(12):
    Y_Month = np.round( R_SST_Historical_12Month_All[month,0], 3)
    Y_Month_Correct = np.round( R_SST_Historical_12Month_Correct_All[month,0], 3)
    Str_Y_Month = str(Y_Month)
    Str_Y_Month_Correct = str(Y_Month_Correct)
    ax.text(X_Month[month] - barwidthnum/2, Y_Month + 0.01, Str_Y_Month, horizontalalignment ='center', fontsize = 10 )
    ax.text(X_Month[month] + barwidthnum/2, Y_Month_Correct + 0.01, Str_Y_Month_Correct, horizontalalignment ='center', fontsize = 10 )
plt.axhline(y = R_SST_Historical_All[0] , c = 'blue', ls = '-', lw = 2)
plt.axhline(y = R_SST_Historical_Correct_All[0] , c = 'red', ls = '-', lw = 2)
Y_Month = np.round(R_SST_Historical_All[0], 3); Str_Y_Month = str(Y_Month)
Y_Month_Correct = np.round(R_SST_Historical_Correct_All[0], 3); Str_Y_Month_Correct = str(Y_Month_Correct)
ax.text(0.3, Y_Month + 0.01, Str_Y_Month, horizontalalignment='center', fontsize = 14)
ax.text(0.3, Y_Month_Correct + 0.01, Str_Y_Month_Correct, horizontalalignment='center', fontsize = 14)
ax.text(0, ylim_R_min + delta_R_bias*0.92, str_panels[1], fontsize=fontsizenum)   # color = 'black'
ax.set(xlim = (0, 13), ylim = (ylim_R_min, ylim_R_max))
ax.tick_params(axis='both', which='both', direction='in')
ax.set_xticks(xticklabel_Month)
ax.set_yticks(yticklabel_R)
ax.set_xticklabels(xticklabels_Month, fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_R, fontsize = fontsizenum)
ax.set_xlabel('Month', fontsize=fontsizenum)
ax.set_ylabel('R',fontsize = fontsizenum)
ax.grid(axis = 'y', linestyle = '--', lw = 1)  # color='grey', alpha=0.5
## 保存图片
fns = os.path.join(Path_SaveFigures, 'Bar_Historical-Correct_12Month.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight')
plt.show()
plt.close()

[2.8733512  3.1363233  3.22852829 2.99000371 2.34460428 1.53561836
 0.88227233 0.46981786 0.44621576 0.88445245 1.57038615 2.30942716]
[0.1161754  0.10373108 0.10248215 0.10359257 0.10119666 0.10398487
 0.10212903 0.1224474  0.10392807 0.11199549 0.11052624 0.12174429]
[[6.26247653e-01 1.12880268e-10]
 [5.90609476e-01 2.15620562e-09]
 [5.53632922e-01 3.22268667e-08]
 [6.23410682e-01 1.44732034e-10]
 [6.29628122e-01 8.36671936e-11]
 [6.02713591e-01 8.24615183e-10]
 [6.88413856e-01 2.39670399e-13]
 [6.54504403e-01 8.21736971e-12]
 [6.31610949e-01 7.00695715e-11]
 [5.55018278e-01 2.92930191e-08]
 [5.68304649e-01 1.14665782e-08]
 [5.84685441e-01 3.40246418e-09]]
[[8.33195725e-01 2.52950835e-23]
 [8.70588812e-01 1.32740289e-27]
 [8.76999319e-01 1.80141553e-28]
 [8.76652973e-01 2.01234975e-28]
 [8.84869577e-01 1.32446015e-29]
 [8.95963924e-01 2.37451200e-31]
 [9.04072536e-01 9.32168081e-33]
 [8.94630190e-01 3.94203824e-31]
 [8.83862539e-01 1.86906514e-29]
 [8.58636844e-01 4.19961008e-26]
 [8

<Figure size 9600x5400 with 2 Axes>

In [16]:
#未来预估的时间序列
#重构的SST：修正前/后，SSP126/245/585
### 预设
fontsizenum = 16  
markersizenum = 4
Num_Year = 2100-2015+1
Num_Sample_SSP = Num_Year*12
X_Time = np.arange(0, Num_Sample_SSP )
Time_SSP = np.array(Time_SSP)
xtick_SSP_interval = 12*12
## SSP126
model_1 = LinearRegression(); model_2 = LinearRegression()
X_Time_2D = X_Time.reshape((-1, 1))
SST_SSP126_2D = SST_SSP126.reshape((-1, 1)); SST_SSP126_Correct_2D = SST_SSP126_Correct_1D.reshape((-1, 1))
model_1.fit(X_Time_2D, SST_SSP126_2D); model_2.fit(X_Time_2D, SST_SSP126_Correct_2D)
Regress_SST_SSP126 = model_1.predict(X_Time_2D); Regress_SST_SSP126_Correct = model_2.predict(X_Time_2D)
xticklabel_SSP = list(range(0, Num_Sample_SSP, xtick_SSP_interval))
print(xticklabel_SSP)
xticklabels_SSP = Time_SSP[xticklabel_SSP[0::]].astype('str').tolist()
ylim_ssp126_max = 22
ylim_ssp126_min = 18
delta_ylim_ssp126 = ylim_ssp126_max - ylim_ssp126_min
yticklabel_SSP126 = np.linspace(ylim_ssp126_min, ylim_ssp126_max, num = 5).tolist()
yticklabel_SSP126 = [round(i,1) for i in yticklabel_SSP126]
print(yticklabel_SSP126)
yticklabels_SSP126 = [str(i) for i in yticklabel_SSP126]
print(yticklabels_SSP126)
## SSP245
model_1 = LinearRegression(); model_2 = LinearRegression()
X_Time_2D = X_Time.reshape((-1, 1))
SST_SSP245_2D = SST_SSP245.reshape((-1, 1)); SST_SSP245_Correct_2D = SST_SSP245_Correct_1D.reshape((-1, 1))
model_1.fit(X_Time_2D, SST_SSP245_2D); model_2.fit(X_Time_2D, SST_SSP245_Correct_2D)
Regress_SST_SSP245 = model_1.predict(X_Time_2D); Regress_SST_SSP245_Correct = model_2.predict(X_Time_2D)
ylim_ssp245_max = 22
ylim_ssp245_min = 18
delta_ylim_ssp245 = ylim_ssp245_max - ylim_ssp245_min
yticklabel_SSP245 = np.linspace(ylim_ssp245_min, ylim_ssp245_max, num = 5).tolist()
yticklabel_SSP245 = [round(i,1) for i in yticklabel_SSP245]
print(yticklabel_SSP245)
yticklabels_SSP245 = [str(i) for i in yticklabel_SSP245]
print(yticklabels_SSP245)
## SSP585
model_1 = LinearRegression(); model_2 = LinearRegression()
X_Time_2D = X_Time.reshape((-1, 1))
SST_SSP585_2D = SST_SSP585.reshape((-1, 1)); SST_SSP585_Correct_2D = SST_SSP585_Correct_1D.reshape((-1, 1))
model_1.fit(X_Time_2D, SST_SSP585_2D); model_2.fit(X_Time_2D, SST_SSP585_Correct_2D)
Regress_SST_SSP585 = model_1.predict(X_Time_2D); Regress_SST_SSP585_Correct = model_2.predict(X_Time_2D)
ylim_ssp585_max = 22
ylim_ssp585_min = 18
delta_ylim_ssp585 = ylim_ssp585_max - ylim_ssp585_min
yticklabel_SSP585 = np.linspace(ylim_ssp585_min, ylim_ssp585_max, num = 5).tolist()
yticklabel_SSP585 = [round(i,1) for i in yticklabel_SSP585]
print(yticklabel_SSP585)
yticklabels_SSP585 = [str(i) for i in yticklabel_SSP585]
print(yticklabels_SSP585)
str_panels = ['a)SSP1-2.6 ', 'b)SSP2-4.5', 'c)SSP5-8.5']

### 画图
fig, ax = plt.subplots(3, 1, figsize = (16,9), dpi = 600)
## SSP126
ax = plt.subplot(3, 1, 1)
plot1, = ax.plot(X_Time, SST_SSP126, 'b-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'blue' )
plot2, = ax.plot(X_Time, SST_SSP126_Correct_1D, 'r-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'red' )
# ax.plot(X_Time, Regress_SST_SSP126.squeeze(), 'b--', linewidth = 2)
# ax.plot(X_Time, Regress_SST_SSP126_Correct.squeeze(), 'r--', linewidth = 2)
ax.text(1, ylim_ssp126_min + delta_ylim_ssp126*0.9, str_panels[0], fontsize = fontsizenum)
ax.set_xlim(0, Num_Sample_SSP)
ax.set_ylim(ylim_ssp126_min, ylim_ssp126_max)  
ax.set_xticks(xticklabel_SSP)
ax.set_yticks(yticklabel_SSP126) 
ax.set_xticklabels([])
ax.set_yticklabels(yticklabels_SSP126 , fontsize = fontsizenum)
# ax.set_xlabel('Time', fontfamily='Times New Roman', fontsize=fontsizenum)
ax.set_ylabel('SST(℃)', fontsize = fontsizenum)
ax.grid(linestyle='-.')
legend_font = { 'weight': 'normal', 'size': fontsizenum}
plt.legend([plot1, plot2], ["FIO-ESM v2", "EEMD-BPNN"], loc = 'upper right', ncol = 1, frameon = False, prop = legend_font) # bbox_to_anchor = (0.3, 1.0),

## SSP245
ax = plt.subplot(3, 1, 2)
ax.plot(X_Time, SST_SSP245, 'b-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'blue' )
ax.plot(X_Time, SST_SSP245_Correct_1D, 'r-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'red' )
ax.text(1, ylim_ssp245_min + delta_ylim_ssp245*0.9, str_panels[1], fontsize = fontsizenum)
ax.set_xlim(0, Num_Sample_SSP)
ax.set_ylim(ylim_ssp245_min, ylim_ssp245_max) 
ax.set_xticks(xticklabel_SSP)
ax.set_yticks(yticklabel_SSP245)  
ax.set_xticklabels([], fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_SSP245, fontsize = fontsizenum)
ax.set_ylabel('SST(℃)', fontsize = fontsizenum)
ax.grid(linestyle='-.')

## SSP585
ax = plt.subplot(3, 1, 3)
ax.plot(X_Time, SST_SSP585, 'b-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'blue' )
ax.plot(X_Time, SST_SSP585_Correct_1D, 'r-', linewidth = 2, marker = '.', markersize = markersizenum, mfc = 'red' )
ax.text(1, ylim_ssp585_min + delta_ylim_ssp585*0.9, str_panels[2], fontsize = fontsizenum)
ax.set_xlim(0, Num_Sample_SSP)
ax.set_ylim(ylim_ssp585_min, ylim_ssp585_max)  
ax.set_xticks(xticklabel_SSP)
ax.set_yticks(yticklabel_SSP585)  
ax.set_xticklabels(xticklabels_SSP, fontsize = fontsizenum)
ax.set_yticklabels(yticklabels_SSP585,  fontsize = fontsizenum)
ax.set_xlabel('Time', fontsize=fontsizenum)
ax.set_ylabel('SST(℃)', fontsize = fontsizenum)
ax.grid(linestyle='-.')
## 保存图片
fns = os.path.join(Path_SaveFigures, 'SST_SSP_Correct.png')
plt.savefig(fns, dpi = 600, bbox_inches = 'tight')
plt.show()
plt.close()

[0, 144, 288, 432, 576, 720, 864, 1008]
[18.0, 19.0, 20.0, 21.0, 22.0]
['18.0', '19.0', '20.0', '21.0', '22.0']
[18.0, 19.0, 20.0, 21.0, 22.0]
['18.0', '19.0', '20.0', '21.0', '22.0']
[18.0, 19.0, 20.0, 21.0, 22.0]
['18.0', '19.0', '20.0', '21.0', '22.0']


<Figure size 9600x5400 with 3 Axes>

In [17]:
# ssp126
SST_ERSST_End20 = SST_ERSST[-20*12::]  # 1995-2014
SST_SSP126_End20 = Regress_SST_SSP126[-20*12::]  # 2081-2100
SST_SSP126_Correct_End20 = Regress_SST_SSP126_Correct[-20*12::]  # 2081-2100
Delta_SST_SSP126 = np.mean(SST_SSP126_End20) - np.mean(SST_ERSST_End20)
Delta_SST_SSP126_Correct = np.mean(SST_SSP126_Correct_End20) - np.mean(SST_ERSST_End20)
print('SSP1-2.6: ','订正前：', Delta_SST_SSP126,'订正后：', Delta_SST_SSP126_Correct)
# ssp245
SST_SSP245_End20 = Regress_SST_SSP245[-20*12::]  # 2081-2100
SST_SSP245_Correct_End20 = Regress_SST_SSP245_Correct[-20*12::]  # 2081-2100
Delta_SST_SSP245 = np.mean(SST_SSP245_End20) - np.mean(SST_ERSST_End20)
Delta_SST_SSP245_Correct = np.mean(SST_SSP245_Correct_End20) - np.mean(SST_ERSST_End20)
print('SSP2-4.5: ','订正前：', Delta_SST_SSP245,'订正后：', Delta_SST_SSP245_Correct)

# ssp585
SST_SSP585_End20 = Regress_SST_SSP585[-20*12::]  # 2081-2100
SST_SSP585_Correct_End20 = Regress_SST_SSP585_Correct[-20*12::]  # 2081-2100
Delta_SST_SSP585 = np.mean(SST_SSP585_End20) - np.mean(SST_ERSST_End20)
Delta_SST_SSP585_Correct = np.mean(SST_SSP585_Correct_End20) - np.mean(SST_ERSST_End20)
print('SSP5-8.5: ','订正前：', Delta_SST_SSP585,'订正后：', Delta_SST_SSP585_Correct)

SSP1-2.6:  订正前： 2.58660499651317 订正后： -0.050160399631852925
SSP2-4.5:  订正前： 3.1505183600805253 订正后： -0.046647489622557714
SSP5-8.5:  订正前： 4.18040185116811 订正后： -0.026628360264123074
